# <center> Model Gini Calculation </center>

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from google.cloud import storage
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

from sklearn.metrics import roc_auc_score
from datetime import datetime, timedelta
# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

# Function

## calculate_gini_for_threedigitscore

In [2]:
# def calculate_gini_for_threedigitscore(scores, labels):
#     """
#     Calculate Gini coefficient for three-digit scores and binary labels
    
#     Parameters:
#     scores: array-like, three-digit scores (higher is better)
#     labels: array-like, binary values (0 or 1, where 1 indicates default)
    
#     Returns:
#     float: Gini coefficient
#     """
#     # Combine scores and labels into a DataFrame
#     df = pd.DataFrame({'score': scores, 'label': labels})
    
#     # Sort by score in descending order (assuming higher score is better)
#     df = df.sort_values('score', ascending=False)
    
#     # Calculate cumulative values
#     total_pos = df['label'].sum()
#     total_neg = len(df) - total_pos
    
#     if total_pos == 0 or total_neg == 0:
#         return 0
    
#     # Calculate cumulative proportions
#     cum_pos = df['label'].cumsum()
#     cum_neg = np.arange(1, len(df) + 1) - cum_pos
    
#     # Convert to proportions
#     cum_pos_prop = cum_pos / total_pos
#     cum_neg_prop = cum_neg / total_neg
    
#     # Calculate Gini
#     gini = 1 - np.trapz(cum_pos_prop, cum_neg_prop)
    
#     return gini


## Modified one

def calculate_gini_for_threedigitscore(scores, labels):
    """
    Calculate Gini coefficient for three-digit scores and binary labels
    
    Parameters:
    scores: array-like, three-digit scores (higher is better)
    labels: array-like, binary values (0 or 1, where 1 indicates default)
    
    Returns:
    float: Gini coefficient
    """
    # Combine scores and labels into a DataFrame
    df = pd.DataFrame({'score': scores, 'label': labels})
    
    # Sort by score in descending order (assuming higher score means lower risk)
    # For default prediction, we want to sort scores in ascending order 
    # since higher default probability should correspond to higher risk
    df = df.sort_values('score', ascending=True)  # Changed to ascending=True
    
    # Calculate cumulative values
    total_pos = df['label'].sum()
    total_neg = len(df) - total_pos
    
    if total_pos == 0 or total_neg == 0:
        return 0
    
    # Calculate cumulative proportions
    cum_pos = df['label'].cumsum()
    cum_neg = np.arange(1, len(df) + 1) - cum_pos
    
    # Convert to proportions
    cum_pos_prop = cum_pos / total_pos
    cum_neg_prop = cum_neg / total_neg
    
    # Calculate area under curve
    auc = np.trapz(cum_pos_prop, cum_neg_prop)
    
    # Calculate Gini
    gini = 2 * auc - 1
    
    return gini

## calculate_gini

In [3]:
def calculate_gini(pd_scores, bad_indicators):
    """
    Calculate Gini coefficient from scores and binary indicators
    
    Parameters:
    pd_scores: array-like of scores/probabilities
    bad_indicators: array-like of binary outcomes (0/1)
    
    Returns:
    float: Gini coefficient
    """
    # Convert inputs to numpy arrays and ensure they're numeric
    pd_scores = np.array(pd_scores, dtype=float)
    bad_indicators = np.array(bad_indicators, dtype=int)
    
    # Check for valid input data
    if len(pd_scores) == 0 or len(bad_indicators) == 0:
        return np.nan
    
    # Check if we have both good and bad cases (needed for ROC AUC)
    if len(np.unique(bad_indicators)) < 2:
        return np.nan
    
    # Calculate AUC using sklearn
    try:
        auc = roc_auc_score(bad_indicators, pd_scores)
        # Calculate Gini from AUC
        gini = 2 * auc - 1
        return gini
    except ValueError:
        return np.nan

## calculate_hybrid_gini

In [4]:
# def calculate_hybrid_gini(scores, labels):
#     """
#     Calculate Gini coefficient handling both PD values and three-digit scores
    
#     Parameters:
#     scores: array-like, contains either PD values (0-1) or three-digit scores
#     labels: array-like, binary values (0 or 1, where 1 indicates default)
    
#     Returns:
#     float: Gini coefficient
#     """
#     # Convert inputs to numpy arrays
#     scores = np.array(scores, dtype=float)
#     labels = np.array(labels, dtype=int)
    
#     # Basic validation
#     if len(scores) == 0 or len(labels) == 0:
#         return np.nan
    
#     if len(np.unique(labels)) < 2:
#         return np.nan
        
#     # Determine if scores are PD values or three-digit scores
#     # PD values are between 0 and 1
#     is_pd = np.all((scores >= 0) & (scores <= 1))
    
#     if is_pd:
#         try:
#             auc = roc_auc_score(labels, scores)
#             gini = 2 * auc - 1
#             return gini
#         except ValueError:
#             return np.nan
#     else:
#         # Handle as three-digit score
#         df = pd.DataFrame({'score': scores, 'label': labels})
#         df = df.sort_values('score', ascending=False)
        
#         total_pos = df['label'].sum()
#         total_neg = len(df) - total_pos
        
#         if total_pos == 0 or total_neg == 0:
#             return np.nan
        
#         cum_pos = df['label'].cumsum()
#         cum_neg = np.arange(1, len(df) + 1) - cum_pos
        
#         cum_pos_prop = cum_pos / total_pos
#         cum_neg_prop = cum_neg / total_neg
        
#         gini = 1 - np.trapz(cum_pos_prop, cum_neg_prop)
#         return gini

## Modified one

def calculate_hybrid_gini(scores, labels):
    """
    Calculate Gini coefficient handling both PD values and three-digit scores
    
    Parameters:
    scores: array-like, contains either PD values (0-1) or three-digit scores
    labels: array-like, binary values (0 or 1, where 1 indicates default)
    
    Returns:
    float: Gini coefficient
    """
    # Convert inputs to numpy arrays
    scores = np.array(scores, dtype=float)
    labels = np.array(labels, dtype=int)
    
    # Basic validation
    if len(scores) == 0 or len(labels) == 0:
        return np.nan
    
    if len(np.unique(labels)) < 2:
        return np.nan
        
    # Determine if scores are PD values or three-digit scores
    # PD values are between 0 and 1
    is_pd = np.all((scores >= 0) & (scores <= 1))
    
    if is_pd:
        try:
            auc = roc_auc_score(labels, scores)
            gini = 2 * auc - 1
            return gini
        except ValueError:
            return np.nan
    else:
        # Handle as three-digit score
        df = pd.DataFrame({'score': scores, 'label': labels})
        # Sort by score in ascending order since higher score means higher risk
        df = df.sort_values('score', ascending=True)
        
        total_pos = df['label'].sum()
        total_neg = len(df) - total_pos
        
        if total_pos == 0 or total_neg == 0:
            return np.nan
        
        cum_pos = df['label'].cumsum()
        cum_neg = np.arange(1, len(df) + 1) - cum_pos
        
        cum_pos_prop = cum_pos / total_pos
        cum_neg_prop = cum_neg / total_neg
        
        # Calculate area under curve
        auc = np.trapz(cum_pos_prop, cum_neg_prop)
        
        # Calculate Gini using the same formula as PD values
        gini = 2 * auc - 1
        return gini

## calculate_periodic_gini_threedigit

In [5]:
# Main processing code
def calculate_periodic_gini_threedigit(df, score_column, label_column, namecolumn):
    """
    Calculate periodic Gini coefficients
    
    Parameters:
    df: DataFrame with disbursement dates and score/label columns
    score_column: name of the score column
    label_column: name of the label column
    """
    # Ensure date is datetime type
    df['disbursementdate'] = pd.to_datetime(df['disbursementdate'])
    
    # Calculate weekly Gini
    df['week'] = df['disbursementdate'].dt.to_period('W')
    weekly_gini = df.groupby('week').apply(
        lambda x: calculate_gini_for_threedigitscore(x[score_column], x[label_column])
    ).reset_index(name='gini')
    weekly_gini['period'] = 'Week'
    weekly_gini['start_date'] = weekly_gini['week'].apply(lambda x: x.to_timestamp())
    weekly_gini['end_date'] = weekly_gini['start_date'] + timedelta(days=6)
    weekly_gini = weekly_gini[['start_date', 'end_date', 'gini', 'period']]
    
    # Calculate monthly Gini
    df['month'] = df['disbursementdate'].dt.to_period('M')
    monthly_gini = df.groupby('month').apply(
        lambda x: calculate_gini_for_threedigitscore(x[score_column], x[label_column])
    ).reset_index(name='gini')
    monthly_gini['period'] = 'Month'
    monthly_gini['start_date'] = monthly_gini['month'].apply(lambda x: x.to_timestamp())
    monthly_gini['end_date'] = monthly_gini['start_date'] + pd.DateOffset(months=1) - pd.Timedelta(days=1)
    monthly_gini = monthly_gini[['start_date', 'end_date', 'gini', 'period']]
    
    # Combine and sort results
    gini_results = pd.concat([weekly_gini, monthly_gini])
    gini_results = gini_results.sort_values(by='start_date').reset_index(drop=True)
    
    # Add metadata columns
    gini_results['Model_Name'] = score_column
    gini_results['version'] = '1.1.0'
    gini_results['bad_rate'] = namecolumn
    gini_results.rename(columns={'gini': f'{score_column}_{namecolumn}_gini'}, inplace=True)
    
    return gini_results

## calculate_periodic_gini

In [6]:
def calculate_periodic_gini(df, score_column, label_column, namecolumn):
    """
    Calculate periodic Gini coefficients
    
    Parameters:
    df: DataFrame with disbursement dates and score/label columns
    score_column: name of the score column
    label_column: name of the label column
    """
    # Input validation
    required_columns = ['disbursementdate', score_column, label_column]
    if not all(col in df.columns for col in required_columns):
        raise ValueError(f"Missing required columns. Need: {required_columns}")
        
    # Create a copy to avoid modifying original dataframe
    df = df.copy()
    
    # Ensure date is datetime type
    df['disbursementdate'] = pd.to_datetime(df['disbursementdate'])
    
    # Ensure score and label columns are numeric
    df[score_column] = pd.to_numeric(df[score_column], errors='coerce')
    df[label_column] = pd.to_numeric(df[label_column], errors='coerce')
    
    # Drop rows with invalid values
    df = df.dropna(subset=[score_column, label_column])
    
    # Calculate weekly Gini
    df['week'] = df['disbursementdate'].dt.to_period('W')
    weekly_gini = df.groupby('week').apply(
        lambda x: calculate_gini(x[score_column], x[label_column])
        if len(x) >= 10 else np.nan  # Only calculate if we have enough samples
    ).reset_index(name='gini')
    weekly_gini['period'] = 'Week'
    weekly_gini['start_date'] = weekly_gini['week'].apply(lambda x: x.to_timestamp())
    weekly_gini['end_date'] = weekly_gini['start_date'] + timedelta(days=6)
    weekly_gini = weekly_gini[['start_date', 'end_date', 'gini', 'period']]
    
    # Calculate monthly Gini
    df['month'] = df['disbursementdate'].dt.to_period('M')
    monthly_gini = df.groupby('month').apply(
        lambda x: calculate_gini(x[score_column], x[label_column])
        if len(x) >= 20 else np.nan  # Only calculate if we have enough samples
    ).reset_index(name='gini')
    monthly_gini['period'] = 'Month'
    monthly_gini['start_date'] = monthly_gini['month'].apply(lambda x: x.to_timestamp())
    monthly_gini['end_date'] = monthly_gini['start_date'] + pd.DateOffset(months=1) - pd.Timedelta(days=1)
    monthly_gini = monthly_gini[['start_date', 'end_date', 'gini', 'period']]
    
    # Combine and sort results
    gini_results = pd.concat([weekly_gini, monthly_gini])
    gini_results = gini_results.sort_values(by='start_date').reset_index(drop=True)
    
    # Add metadata columns
    gini_results['Model_Name'] = score_column
    gini_results['version'] = '1.1.0'
    gini_results['bad_rate'] = namecolumn
    gini_results.rename(columns={'gini': f'{score_column}_{namecolumn}_gini'}, inplace=True)
    
    return gini_results

## calculate_periodic_hybrid_gini

In [7]:
def calculate_periodic_hybrid_gini(df, score_column, label_column, namecolumn):
    """
    Calculate periodic Gini coefficients for mixed score types
    
    Parameters:
    df: DataFrame with disbursement dates and score/label columns
    score_column: name of the score column
    label_column: name of the label column
    """
    # Input validation
    required_columns = ['disbursementdate', score_column, label_column]
    if not all(col in df.columns for col in required_columns):
        raise ValueError(f"Missing required columns. Need: {required_columns}")
        
    # Create a copy to avoid modifying original dataframe
    df = df.copy()
    
    # Ensure date is datetime type
    df['disbursementdate'] = pd.to_datetime(df['disbursementdate'])
    
    # Ensure score and label columns are numeric
    df[score_column] = pd.to_numeric(df[score_column], errors='coerce')
    df[label_column] = pd.to_numeric(df[label_column], errors='coerce')
    
    # Drop rows with invalid values
    df = df.dropna(subset=[score_column, label_column])
    
    # Calculate weekly Gini
    df['week'] = df['disbursementdate'].dt.to_period('W')
    weekly_gini = df.groupby('week').apply(
        lambda x: calculate_hybrid_gini(x[score_column], x[label_column])
        if len(x) >= 10 else np.nan  # Only calculate if we have enough samples
    ).reset_index(name='gini')
    weekly_gini['period'] = 'Week'
    weekly_gini['start_date'] = weekly_gini['week'].apply(lambda x: x.to_timestamp())
    weekly_gini['end_date'] = weekly_gini['start_date'] + pd.Timedelta(days=6)
    weekly_gini = weekly_gini[['start_date', 'end_date', 'gini', 'period']]
    
    # Calculate monthly Gini
    df['month'] = df['disbursementdate'].dt.to_period('M')
    monthly_gini = df.groupby('month').apply(
        lambda x: calculate_hybrid_gini(x[score_column], x[label_column])
        if len(x) >= 20 else np.nan  # Only calculate if we have enough samples
    ).reset_index(name='gini')
    monthly_gini['period'] = 'Month'
    monthly_gini['start_date'] = monthly_gini['month'].apply(lambda x: x.to_timestamp())
    monthly_gini['end_date'] = monthly_gini['start_date'] + pd.DateOffset(months=1) - pd.Timedelta(days=1)
    monthly_gini = monthly_gini[['start_date', 'end_date', 'gini', 'period']]
    
    # Combine and sort results
    gini_results = pd.concat([weekly_gini, monthly_gini])
    gini_results = gini_results.sort_values(by='start_date').reset_index(drop=True)
    
    # Add metadata columns
    gini_results['Model_Name'] = score_column
    gini_results['version'] = '1.1.0'
    gini_results['bad_rate'] = namecolumn
    gini_results.rename(columns={'gini': f'{score_column}_{namecolumn}_gini'}, inplace=True)
    
    return gini_results

# App Score FPD10

In [8]:
sq = """with appscore as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    s_apps_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    s_apps_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from appscore;"""

dfappscorefpd10 = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID 1c55dd71-8e73-469d-9f98-aa973bb4ce5d successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [9]:
dfappscorefpd10.sample(5)

,disbursementdate,digitalLoanAccountId,s_apps_score,ln_fpd10_flag,ln_mature_fpd10_flag
95467,2024-06-23 12:11:15,1965b8de-6c58-4a71-9357-6bba107ac15b,0.542696,0,1
136769,2024-07-21 16:17:36,00987e5a-744f-4911-8187-3e6f6baaa6ed,0.430678,0,1
150193,2024-08-21 15:45:56,1347a9b4-8baa-4618-86ca-73fb4fef34e9,0.671854,1,1
57728,2024-06-02 09:53:16,bcbefb9d-8468-4fcd-beab-6f26c9d7a174,0.423371,0,1
5187,2024-02-25 18:12:29,1b401822-89b1-44ad-b866-854feff8af07,0.445916,1,1


In [10]:
gini_results = calculate_periodic_gini(dfappscorefpd10, 's_apps_score', 'ln_fpd10_flag', 'FPD10')

In [11]:
gini_results.head()

,start_date,end_date,s_apps_score_FPD10_gini,period,Model_Name,version,bad_rate
0,2023-05-29,2023-06-04,0.548230,Week,s_apps_score,1.1.0,FPD10
1,2023-06-01,2023-06-30,0.384648,Month,s_apps_score,1.1.0,FPD10
2,2023-06-05,2023-06-11,0.500000,Week,s_apps_score,1.1.0,FPD10
3,2023-06-12,2023-06-18,0.555195,Week,s_apps_score,1.1.0,FPD10
4,2023-06-19,2023-06-25,0.004831,Week,s_apps_score,1.1.0,FPD10


In [12]:
appscoreFPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{appscoreFPD10.shape}")
appscoreFPD10.columns.values

The shape of dataframe after copy is:	(106, 7)


array(['start_date', 'end_date', 's_apps_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [13]:
gini_results.head()

,start_date,end_date,s_apps_score_FPD10_gini,period,Model_Name,version,bad_rate
0,2023-05-29,2023-06-04,0.548230,Week,s_apps_score,1.1.0,FPD10
1,2023-06-01,2023-06-30,0.384648,Month,s_apps_score,1.1.0,FPD10
2,2023-06-05,2023-06-11,0.500000,Week,s_apps_score,1.1.0,FPD10
3,2023-06-12,2023-06-18,0.555195,Week,s_apps_score,1.1.0,FPD10
4,2023-06-19,2023-06-25,0.004831,Week,s_apps_score,1.1.0,FPD10


# App Score FPD30

In [14]:
sq = """
with appscore as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    s_apps_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    s_apps_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from appscore;
"""

dfappscorefpd30 = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID 68efee5c-52f7-4bd4-89bc-b2b1cbb9140f successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [15]:
dfappscorefpd30.head()

,disbursementdate,digitalLoanAccountId,s_apps_score,ln_fpd30_flag,ln_mature_fpd30_flag
0,2023-06-01 10:40:18,ac002041-b0be-446e-b173-65effe340651,0.601410,0,1
1,2023-06-01 12:56:23,e585b35d-bc13-494d-970d-2272d976991b,0.411007,0,1
2,2023-06-01 15:36:13,a3dc185d-0118-4ebc-bf03-117fdd8f9c64,0.550104,0,1
3,2023-06-01 17:17:21,3b2e2242-085e-4948-83f4-1a48bcb1b67b,0.339053,0,1
4,2023-06-01 18:40:13,ab22972c-29cf-4209-967c-3af7b19f382d,0.355267,0,1


In [16]:
gini_results = calculate_periodic_gini(dfappscorefpd30, 's_apps_score', 'ln_fpd30_flag', 'FPD30')
# gini_results['bad_rate'] = 'FPD30'
appscoreFPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{appscoreFPD30.shape}")
appscoreFPD30.columns.values

The shape of dataframe after copy is:	(102, 7)


array(['start_date', 'end_date', 's_apps_score_FPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [17]:
gini_results.sample(5)

,start_date,end_date,s_apps_score_FPD30_gini,period,Model_Name,version,bad_rate
64,2024-05-27,2024-06-02,0.438049,Week,s_apps_score,1.1.0,FPD30
43,2024-01-29,2024-02-04,0.459499,Week,s_apps_score,1.1.0,FPD30
28,2023-11-01,2023-11-30,0.309397,Month,s_apps_score,1.1.0,FPD30
29,2023-11-06,2023-11-12,0.175815,Week,s_apps_score,1.1.0,FPD30
56,2024-04-08,2024-04-14,0.475546,Week,s_apps_score,1.1.0,FPD30


# App Score FSPD30

In [18]:
sq = """with appscore as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    s_apps_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    s_apps_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from appscore;
"""

dfappscorefspd30 = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID 32e22513-4289-4a9b-afcd-ec705cdc179b successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [19]:
gini_results = calculate_periodic_gini(dfappscorefspd30, 's_apps_score', 'ln_fspd30_flag', 'FSPD30')
# gini_results['bad_rate'] = 'FSPD30'
appscoreFSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{appscoreFSPD30.shape}")
appscoreFSPD30.columns.values

The shape of dataframe after copy is:	(97, 7)


array(['start_date', 'end_date', 's_apps_score_FSPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [20]:
gini_results.tail()

,start_date,end_date,s_apps_score_FSPD30_gini,period,Model_Name,version,bad_rate
92,2024-11-01,2024-11-30,0.298595,Month,s_apps_score,1.1.0,FSPD30
93,2024-11-04,2024-11-10,0.295698,Week,s_apps_score,1.1.0,FSPD30
94,2024-11-11,2024-11-17,0.306730,Week,s_apps_score,1.1.0,FSPD30
95,2024-11-18,2024-11-24,0.277736,Week,s_apps_score,1.1.0,FSPD30
96,2024-11-25,2024-12-01,0.245972,Week,s_apps_score,1.1.0,FSPD30


# App Score FSTPD30

In [21]:
sq = """with appscore as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    s_apps_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    s_apps_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from appscore;
"""

dfappscorefstpd30 = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID 4842e6c1-4c06-4f8a-ba5e-1fc83914ac5b successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [22]:
gini_results = calculate_periodic_gini(dfappscorefstpd30, 's_apps_score', 'ln_fstpd30_flag', 'FSTPD30')
# gini_results['bad_rate'] = 'FSTPD30'
appscoreFSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{appscoreFSTPD30.shape}")
appscoreFSTPD30.columns.values

The shape of dataframe after copy is:	(91, 7)


array(['start_date', 'end_date', 's_apps_score_FSTPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [23]:
appscoreFSTPD30.head()

,start_date,end_date,s_apps_score_FSTPD30_gini,period,Model_Name,version,bad_rate
0,2023-05-29,2023-06-04,0.465098,Week,s_apps_score,1.1.0,FSTPD30
1,2023-06-01,2023-06-30,0.337420,Month,s_apps_score,1.1.0,FSTPD30
2,2023-06-05,2023-06-11,0.413300,Week,s_apps_score,1.1.0,FSTPD30
3,2023-06-12,2023-06-18,0.395676,Week,s_apps_score,1.1.0,FSTPD30
4,2023-06-19,2023-06-25,0.314497,Week,s_apps_score,1.1.0,FSTPD30


# Combining App Score

In [24]:
import functools

dataframes = [appscoreFPD10, appscoreFPD30, appscoreFSPD30, appscoreFSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 's_apps_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 's_apps_score_FPD30_gini',
       's_apps_score_FSPD30_gini', 's_apps_score_FSTPD30_gini'],
      dtype=object)

In [25]:
final_df = final_df[['start_date', 'end_date', 'period',   'Model_Name', 'version', 'bad_rate','s_apps_score_FSTPD30_gini','s_apps_score_FSPD30_gini',
       's_apps_score_FPD30_gini', 's_apps_score_FPD10_gini']].copy()
final_df.dtypes

start_date                   datetime64[ns]
end_date                     datetime64[ns]
period                               object
Model_Name                           object
version                              object
bad_rate                             object
s_apps_score_FSTPD30_gini           float64
s_apps_score_FSPD30_gini            float64
s_apps_score_FPD30_gini             float64
s_apps_score_FPD10_gini             float64
dtype: object

## Creating app score table 

In [26]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_s_apps_score;"""
client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=0c437e58-e5c8-4f1f-b3b8-ea8e198e334c>

In [27]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('s_apps_score_FSTPD30_gini', 'FLOAT'),
    bigquery.SchemaField('s_apps_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('s_apps_score_FPD30_gini', 'FLOAT'),
    bigquery.SchemaField('s_apps_score_fpd10_gini', 'FLOAT')
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_s_apps_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=1b6638d6-267a-451a-8902-647f80b3afa6>

# sb_demo_score

## FPD10

In [28]:
sq = """
with sb_demo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    sb_demo_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    sb_demo_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from sb_demo_score;
"""

df_sb_demo_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID a0327a74-d490-45fc-8b9f-1c79f2464d19 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [29]:
df_sb_demo_scorefpd10.head()

,disbursementdate,digitalLoanAccountId,sb_demo_score,ln_fpd10_flag,ln_mature_fpd10_flag
0,2023-06-01 10:40:18,ac002041-b0be-446e-b173-65effe340651,0.173099,0,1
1,2023-06-01 12:56:23,e585b35d-bc13-494d-970d-2272d976991b,0.113972,0,1
2,2023-06-01 15:36:13,a3dc185d-0118-4ebc-bf03-117fdd8f9c64,0.134361,0,1
3,2023-06-01 17:17:21,3b2e2242-085e-4948-83f4-1a48bcb1b67b,0.140578,0,1
4,2023-06-01 18:40:13,ab22972c-29cf-4209-967c-3af7b19f382d,0.252004,0,1


In [30]:
gini_results = calculate_periodic_gini(df_sb_demo_scorefpd10, 'sb_demo_score', 'ln_fpd10_flag', 'FPD10')
sb_demo_scoreFPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{sb_demo_scoreFPD10.shape}")
sb_demo_scoreFPD10.columns.values

The shape of dataframe after copy is:	(106, 7)


array(['start_date', 'end_date', 'sb_demo_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [31]:
sb_demo_scoreFPD10.tail()

,start_date,end_date,sb_demo_score_FPD10_gini,period,Model_Name,version,bad_rate
101,2024-12-23,2024-12-29,0.235289,Week,sb_demo_score,1.1.0,FPD10
102,2024-12-30,2025-01-05,0.257868,Week,sb_demo_score,1.1.0,FPD10
103,2025-01-01,2025-01-31,0.230445,Month,sb_demo_score,1.1.0,FPD10
104,2025-01-06,2025-01-12,0.166198,Week,sb_demo_score,1.1.0,FPD10
105,2025-01-13,2025-01-19,0.272316,Week,sb_demo_score,1.1.0,FPD10


## FPD30

In [32]:
sq = """
with sb_demo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    sb_demo_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    sb_demo_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from sb_demo_score;
"""

df_sb_demo_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID 1260313a-e001-4d61-9589-6912006f6082 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [33]:
gini_results = calculate_periodic_gini(df_sb_demo_scorefpd30, 'sb_demo_score', 'ln_fpd30_flag', 'FPD30')
sb_demo_scoreFPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{sb_demo_scoreFPD30.shape}")
sb_demo_scoreFPD30.columns.values

The shape of dataframe after copy is:	(102, 7)


array(['start_date', 'end_date', 'sb_demo_score_FPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [34]:
sb_demo_scoreFPD30.head() 

,start_date,end_date,sb_demo_score_FPD30_gini,period,Model_Name,version,bad_rate
0,2023-05-29,2023-06-04,0.330909,Week,sb_demo_score,1.1.0,FPD30
1,2023-06-01,2023-06-30,0.189531,Month,sb_demo_score,1.1.0,FPD30
2,2023-06-05,2023-06-11,0.130370,Week,sb_demo_score,1.1.0,FPD30
3,2023-06-12,2023-06-18,0.371746,Week,sb_demo_score,1.1.0,FPD30
4,2023-06-19,2023-06-25,-0.052209,Week,sb_demo_score,1.1.0,FPD30


## FSPD30

In [35]:
sq = """
with sb_demo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    sb_demo_score,
    ln_fspd30_flag,   -- fspd30
	ln_mature_fspd30_flag,	--- fspd30 observation
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    sb_demo_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from sb_demo_score;
"""

df_sb_demo_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID 773a7173-1af9-485b-84d1-1d8d6c7b5b0a successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [36]:
gini_results = calculate_periodic_gini(df_sb_demo_scorefspd30, 'sb_demo_score', 'ln_fspd30_flag', 'FSPD30')
sb_demo_scoreFSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{sb_demo_scoreFSPD30.shape}")
sb_demo_scoreFSPD30.columns.values

The shape of dataframe after copy is:	(97, 7)


array(['start_date', 'end_date', 'sb_demo_score_FSPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [37]:
sb_demo_scoreFSPD30.head()

,start_date,end_date,sb_demo_score_FSPD30_gini,period,Model_Name,version,bad_rate
0,2023-05-29,2023-06-04,0.098383,Week,sb_demo_score,1.1.0,FSPD30
1,2023-06-01,2023-06-30,0.184759,Month,sb_demo_score,1.1.0,FSPD30
2,2023-06-05,2023-06-11,0.239596,Week,sb_demo_score,1.1.0,FSPD30
3,2023-06-12,2023-06-18,0.312568,Week,sb_demo_score,1.1.0,FSPD30
4,2023-06-19,2023-06-25,0.010796,Week,sb_demo_score,1.1.0,FSPD30


## FSTPD30

In [38]:
sq = """
with sb_demo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    sb_demo_score,
    ln_fstpd30_flag,   -- fstpd30
	ln_mature_fstpd30_flag,	--- fstpd30 observation
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    sb_demo_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from sb_demo_score;
"""

df_sb_demo_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID cf35d5ff-86ec-4420-8d08-fa2f0c05e28d successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [39]:
gini_results = calculate_periodic_gini(df_sb_demo_scorefstpd30, 'sb_demo_score', 'ln_fstpd30_flag', 'FSTPD30')
sb_demo_scoreFSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{sb_demo_scoreFSTPD30.shape}")
sb_demo_scoreFSTPD30.columns.values

The shape of dataframe after copy is:	(91, 7)


array(['start_date', 'end_date', 'sb_demo_score_FSTPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [40]:
sb_demo_scoreFSTPD30.head()

,start_date,end_date,sb_demo_score_FSTPD30_gini,period,Model_Name,version,bad_rate
0,2023-05-29,2023-06-04,0.215686,Week,sb_demo_score,1.1.0,FSTPD30
1,2023-06-01,2023-06-30,0.165765,Month,sb_demo_score,1.1.0,FSTPD30
2,2023-06-05,2023-06-11,0.281272,Week,sb_demo_score,1.1.0,FSTPD30
3,2023-06-12,2023-06-18,0.142612,Week,sb_demo_score,1.1.0,FSTPD30
4,2023-06-19,2023-06-25,0.096533,Week,sb_demo_score,1.1.0,FSTPD30


# Combining sb demo score

In [41]:
import functools

dataframes = [sb_demo_scoreFPD10, sb_demo_scoreFPD30, sb_demo_scoreFSPD30, sb_demo_scoreFSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'sb_demo_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 'sb_demo_score_FPD30_gini',
       'sb_demo_score_FSPD30_gini', 'sb_demo_score_FSTPD30_gini'],
      dtype=object)

In [42]:
final_df = final_df[['start_date', 'end_date', 'period',
       'Model_Name', 'version', 'bad_rate','sb_demo_score_FPD10_gini','sb_demo_score_FPD30_gini',
       'sb_demo_score_FSPD30_gini', 'sb_demo_score_FSTPD30_gini']].copy()

## creating sb demo score table 

In [43]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_sb_demo_score;"""
client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=f50c16d8-ab07-47a2-b4bc-a3bbffb5ffad>

In [44]:


import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('sb_demo_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('sb_demo_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('sb_demo_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('sb_demo_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_sb_demo_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=d92c3926-af71-4fb4-ad47-4f0ec500d85d>

# s_cic_score

## FPD10

In [45]:
sq = """
with s_cic_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    s_cic_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    s_cic_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from s_cic_score;
"""

df_s_cic_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID dda6223e-185c-4c6f-aab8-7488e907450b successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [46]:
df_s_cic_scorefpd10.head()

,disbursementdate,digitalLoanAccountId,s_cic_score,ln_fpd10_flag,ln_mature_fpd10_flag
0,2023-06-02 15:33:58,531e98fe-749e-471e-9960-6dd80d55e86f,0.153677,1,1
1,2023-06-02 16:25:26,5dae7da0-f426-4402-b552-f84ab33d38bf,0.125483,1,1
2,2023-06-02 17:11:42,1d7abbe4-a62a-4f4b-91b2-bb360b9e61c2,0.170078,0,1
3,2023-06-03 11:29:31,482aba08-848c-4ce4-a4f2-6680e85d4e5d,0.079289,0,1
4,2023-06-03 11:35:43,5e3ca494-7918-406b-b437-a76164ff4ba6,0.071766,0,1


In [47]:
gini_results = calculate_periodic_gini(df_s_cic_scorefpd10, 's_cic_score', 'ln_fpd10_flag', 'FPD10')
s_cic_scoreFPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{s_cic_scoreFPD10.shape}")
s_cic_scoreFPD10.columns.values

The shape of dataframe after copy is:	(105, 7)


array(['start_date', 'end_date', 's_cic_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [48]:
s_cic_scoreFPD10.tail()

,start_date,end_date,s_cic_score_FPD10_gini,period,Model_Name,version,bad_rate
100,2024-12-23,2024-12-29,0.181444,Week,s_cic_score,1.1.0,FPD10
101,2024-12-30,2025-01-05,0.179916,Week,s_cic_score,1.1.0,FPD10
102,2025-01-01,2025-01-31,0.164863,Month,s_cic_score,1.1.0,FPD10
103,2025-01-06,2025-01-12,0.169427,Week,s_cic_score,1.1.0,FPD10
104,2025-01-13,2025-01-19,-0.007814,Week,s_cic_score,1.1.0,FPD10


## FPD30

In [49]:
sq = """
with s_cic_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    s_cic_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    s_cic_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from s_cic_score;
"""

df_s_cic_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID 4c1fb4f0-019a-48b9-aed1-7782d68ad8f0 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [50]:
df_s_cic_scorefpd30.head()

,disbursementdate,digitalLoanAccountId,s_cic_score,ln_fpd30_flag,ln_mature_fpd30_flag
0,2024-12-22 12:23:28,64ae039c-0cbe-47c0-9dec-5133167fa4fe,0.059829,0,1
1,2024-12-14 11:40:28,718b7f93-8bdf-47cd-8866-6b6768677989,0.089102,0,1
2,2024-12-21 17:00:39,5911a7ab-9836-466f-a6f5-e6561195358f,0.104869,0,1
3,2024-12-06 12:32:01,683da554-bdd7-4657-bec6-6771f9ecaae6,0.199273,0,1
4,2024-12-03 17:00:38,ff061c29-845b-42fc-9acd-119bbc47b920,0.126366,0,1


In [51]:
gini_results = calculate_periodic_gini(df_s_cic_scorefpd30, 's_cic_score', 'ln_fpd30_flag', 'FPD30')
s_cic_scoreFPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{s_cic_scoreFPD30.shape}")
s_cic_scoreFPD30.columns.values

The shape of dataframe after copy is:	(101, 7)


array(['start_date', 'end_date', 's_cic_score_FPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [52]:
s_cic_scoreFPD30.head()

,start_date,end_date,s_cic_score_FPD30_gini,period,Model_Name,version,bad_rate
0,2023-05-29,2023-06-04,0.261538,Week,s_cic_score,1.1.0,FPD30
1,2023-06-01,2023-06-30,0.236830,Month,s_cic_score,1.1.0,FPD30
2,2023-06-05,2023-06-11,0.196970,Week,s_cic_score,1.1.0,FPD30
3,2023-06-12,2023-06-18,-0.193069,Week,s_cic_score,1.1.0,FPD30
4,2023-06-19,2023-06-25,0.435484,Week,s_cic_score,1.1.0,FPD30


In [53]:
s_cic_scoreFPD30.describe()

,start_date,end_date,s_cic_score_FPD30_gini
count,101,101,101.000000
mean,2024-03-11 12:21:23.168316928,2024-03-21 22:34:27.326732800,0.262726
min,2023-05-29 00:00:00,2023-06-04 00:00:00,-0.193069
25%,2023-10-23 00:00:00,2023-10-31 00:00:00,0.201149
50%,2024-03-11 00:00:00,2024-03-24 00:00:00,0.268382
75%,2024-08-01 00:00:00,2024-08-11 00:00:00,0.325877
max,2024-12-23 00:00:00,2024-12-31 00:00:00,0.749150
std,NaN,NaN,0.121692


## FSPD30

In [54]:
sq = """
with s_cic_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    s_cic_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    s_cic_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from s_cic_score;
"""

df_s_cic_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID 1ccc0be7-98cd-4910-8ffd-907a7f2aca2a successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [55]:
gini_results = calculate_periodic_gini(df_s_cic_scorefspd30, 's_cic_score', 'ln_fspd30_flag', 'FSPD30')
s_cic_scoreFSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{s_cic_scoreFSPD30.shape}")
s_cic_scoreFSPD30.columns.values

The shape of dataframe after copy is:	(96, 7)


array(['start_date', 'end_date', 's_cic_score_FSPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [56]:
s_cic_scoreFSPD30.describe()

,start_date,end_date,s_cic_score_FSPD30_gini
count,96,96,96.000000
mean,2024-02-26 06:15:00,2024-03-07 16:00:00,0.264402
min,2023-05-29 00:00:00,2023-06-04 00:00:00,0.040799
25%,2023-10-14 06:00:00,2023-10-27 06:00:00,0.209456
50%,2024-02-28 00:00:00,2024-03-06 12:00:00,0.270390
75%,2024-07-09 18:00:00,2024-07-22 18:00:00,0.312949
max,2024-11-25 00:00:00,2024-12-01 00:00:00,0.545299
std,NaN,NaN,0.082780


In [57]:
s_cic_scoreFSPD30.tail()

,start_date,end_date,s_cic_score_FSPD30_gini,period,Model_Name,version,bad_rate
91,2024-11-01,2024-11-30,0.249632,Month,s_cic_score,1.1.0,FSPD30
92,2024-11-04,2024-11-10,0.317348,Week,s_cic_score,1.1.0,FSPD30
93,2024-11-11,2024-11-17,0.199503,Week,s_cic_score,1.1.0,FSPD30
94,2024-11-18,2024-11-24,0.255737,Week,s_cic_score,1.1.0,FSPD30
95,2024-11-25,2024-12-01,0.040799,Week,s_cic_score,1.1.0,FSPD30


## FSTPD30

In [58]:
sq = """
with s_cic_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    s_cic_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    s_cic_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from s_cic_score;
"""

df_s_cic_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID 9a49c253-2825-48cd-b9a7-202517310866 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [59]:
gini_results = calculate_periodic_gini(df_s_cic_scorefstpd30, 's_cic_score', 'ln_fstpd30_flag', 'FSTPD30')
s_cic_scoreFSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{s_cic_scoreFSTPD30.shape}")
s_cic_scoreFSTPD30.columns.values

The shape of dataframe after copy is:	(90, 7)


array(['start_date', 'end_date', 's_cic_score_FSTPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [60]:
s_cic_scoreFSTPD30.describe()

,start_date,end_date,s_cic_score_FSTPD30_gini
count,90,90,90.000000
mean,2024-02-09 02:56:00,2024-02-19 13:36:00,0.252032
min,2023-05-29 00:00:00,2023-06-04 00:00:00,0.036643
25%,2023-10-03 18:00:00,2023-10-16 18:00:00,0.212316
50%,2024-02-08 12:00:00,2024-02-21 12:00:00,0.261071
75%,2024-06-15 06:00:00,2024-06-28 06:00:00,0.303623
max,2024-10-21 00:00:00,2024-10-31 00:00:00,0.479082
std,NaN,NaN,0.078320


In [61]:
s_cic_scoreFSTPD30.tail()

,start_date,end_date,s_cic_score_FSTPD30_gini,period,Model_Name,version,bad_rate
85,2024-09-30,2024-10-06,0.276798,Week,s_cic_score,1.1.0,FSTPD30
86,2024-10-01,2024-10-31,0.277683,Month,s_cic_score,1.1.0,FSTPD30
87,2024-10-07,2024-10-13,0.282088,Week,s_cic_score,1.1.0,FSTPD30
88,2024-10-14,2024-10-20,0.283938,Week,s_cic_score,1.1.0,FSTPD30
89,2024-10-21,2024-10-27,0.289494,Week,s_cic_score,1.1.0,FSTPD30


# Combining s_cic_score

In [62]:
import functools

dataframes = [s_cic_scoreFPD10, s_cic_scoreFPD30, s_cic_scoreFSPD30, s_cic_scoreFSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 's_cic_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 's_cic_score_FPD30_gini',
       's_cic_score_FSPD30_gini', 's_cic_score_FSTPD30_gini'],
      dtype=object)

In [63]:
final_df = final_df[['start_date', 'end_date', 'period',
       'Model_Name', 'version', 'bad_rate','s_cic_score_FPD10_gini','s_cic_score_FPD30_gini', 's_cic_score_FSPD30_gini', 's_cic_score_FSTPD30_gini']].copy()

## Creating the table

In [64]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_s_cic_score;"""

client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=74b101be-3aad-471a-99a8-799d2c385a8b>

In [65]:


import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('s_cic_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('s_cic_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('s_cic_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('s_cic_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_s_cic_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=daa7a2dd-53bf-4142-96c6-f62fe95c699d>

# sb_stack_score

## FPD10

In [66]:
# sb_stack_score

sq = """
with sb_stack_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    sb_stack_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    sb_stack_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from sb_stack_score;
"""

df_sb_stack_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

df_sb_stack_scorefpd10.head()

Job ID 8ecac716-fe90-4f50-ba59-f62705e93de2 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


,disbursementdate,digitalLoanAccountId,sb_stack_score,ln_fpd10_flag,ln_mature_fpd10_flag
0,2023-06-01 10:40:18,ac002041-b0be-446e-b173-65effe340651,0.129992,0,1
1,2023-06-01 12:56:23,e585b35d-bc13-494d-970d-2272d976991b,0.043838,0,1
2,2023-06-01 15:36:13,a3dc185d-0118-4ebc-bf03-117fdd8f9c64,0.087192,0,1
3,2023-06-01 17:17:21,3b2e2242-085e-4948-83f4-1a48bcb1b67b,0.022913,0,1
4,2023-06-01 18:40:13,ab22972c-29cf-4209-967c-3af7b19f382d,0.052938,0,1


In [67]:
gini_results = calculate_periodic_gini(df_sb_stack_scorefpd10, 'sb_stack_score', 'ln_fpd10_flag', 'FPD10')
M1FPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M1FPD10.shape}")
M1FPD10.columns.values

The shape of dataframe after copy is:	(106, 7)


array(['start_date', 'end_date', 'sb_stack_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [68]:
M1FPD10.describe()

,start_date,end_date,sb_stack_score_FPD10_gini
count,106,106,106.000000
mean,2024-03-20 14:56:36.226415104,2024-03-31 01:35:05.660377344,0.349183
min,2023-05-29 00:00:00,2023-06-04 00:00:00,-0.067879
25%,2023-10-24 18:00:00,2023-11-01 06:00:00,0.301373
50%,2024-03-21 12:00:00,2024-03-31 00:00:00,0.333806
75%,2024-08-17 06:00:00,2024-08-29 12:00:00,0.411849
max,2025-01-13 00:00:00,2025-01-31 00:00:00,0.659591
std,NaN,NaN,0.097140


In [69]:
M1FPD10.head()

,start_date,end_date,sb_stack_score_FPD10_gini,period,Model_Name,version,bad_rate
0,2023-05-29,2023-06-04,0.482902,Week,sb_stack_score,1.1.0,FPD10
1,2023-06-01,2023-06-30,0.318382,Month,sb_stack_score,1.1.0,FPD10
2,2023-06-05,2023-06-11,0.417052,Week,sb_stack_score,1.1.0,FPD10
3,2023-06-12,2023-06-18,0.659591,Week,sb_stack_score,1.1.0,FPD10
4,2023-06-19,2023-06-25,-0.067879,Week,sb_stack_score,1.1.0,FPD10


## FPD30

In [70]:
# sb_stack_score

sq = """
with sb_stack_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    sb_stack_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    sb_stack_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from sb_stack_score;
"""

df_sb_stack_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

df_sb_stack_scorefpd30.head()

Job ID 4cb8df61-aa3a-4ca3-84df-669cfef5f9ba successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


,disbursementdate,digitalLoanAccountId,sb_stack_score,ln_fpd30_flag,ln_mature_fpd30_flag
0,2023-06-01 10:40:18,ac002041-b0be-446e-b173-65effe340651,0.129992,0,1
1,2023-06-01 12:56:23,e585b35d-bc13-494d-970d-2272d976991b,0.043838,0,1
2,2023-06-01 15:36:13,a3dc185d-0118-4ebc-bf03-117fdd8f9c64,0.087192,0,1
3,2023-06-01 17:17:21,3b2e2242-085e-4948-83f4-1a48bcb1b67b,0.022913,0,1
4,2023-06-01 18:40:13,ab22972c-29cf-4209-967c-3af7b19f382d,0.052938,0,1


In [71]:
gini_results = calculate_periodic_gini(df_sb_stack_scorefpd30, 'sb_stack_score', 'ln_fpd30_flag', 'FPD30')
M2FPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M2FPD30.shape}")
M2FPD30.columns.values

The shape of dataframe after copy is:	(102, 7)


array(['start_date', 'end_date', 'sb_stack_score_FPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [72]:
M2FPD30.describe()

,start_date,end_date,sb_stack_score_FPD30_gini
count,102,102,102.000000
mean,2024-03-09 05:52:56.470588160,2024-03-19 15:03:31.764705792,0.370856
min,2023-05-29 00:00:00,2023-06-04 00:00:00,-0.013387
25%,2023-10-17 18:00:00,2023-10-29 12:00:00,0.324190
50%,2024-03-07 12:00:00,2024-03-20 12:00:00,0.363071
75%,2024-07-31 06:00:00,2024-08-09 06:00:00,0.428887
max,2024-12-23 00:00:00,2024-12-31 00:00:00,0.636364
std,NaN,NaN,0.096995


In [73]:
M2FPD30.tail()

,start_date,end_date,sb_stack_score_FPD30_gini,period,Model_Name,version,bad_rate
97,2024-12-01,2024-12-31,0.328661,Month,sb_stack_score,1.1.0,FPD30
98,2024-12-02,2024-12-08,0.370326,Week,sb_stack_score,1.1.0,FPD30
99,2024-12-09,2024-12-15,0.301145,Week,sb_stack_score,1.1.0,FPD30
100,2024-12-16,2024-12-22,0.325655,Week,sb_stack_score,1.1.0,FPD30
101,2024-12-23,2024-12-29,0.293112,Week,sb_stack_score,1.1.0,FPD30


## FSPD30

In [74]:
# sb_stack_score

sq = """
with sb_stack_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    sb_stack_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    sb_stack_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from sb_stack_score;
"""

df_sb_stack_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

df_sb_stack_scorefspd30.head()

Job ID df23724d-7614-4792-b3c2-42cf0c56e33a successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


,disbursementdate,digitalLoanAccountId,sb_stack_score,ln_fspd30_flag,ln_mature_fspd30_flag
0,2023-06-01 10:40:18,ac002041-b0be-446e-b173-65effe340651,0.129992,0,1
1,2023-06-01 12:56:23,e585b35d-bc13-494d-970d-2272d976991b,0.043838,0,1
2,2023-06-01 15:36:13,a3dc185d-0118-4ebc-bf03-117fdd8f9c64,0.087192,0,1
3,2023-06-01 17:17:21,3b2e2242-085e-4948-83f4-1a48bcb1b67b,0.022913,0,1
4,2023-06-01 18:40:13,ab22972c-29cf-4209-967c-3af7b19f382d,0.052938,0,1


In [75]:
gini_results = calculate_periodic_gini(df_sb_stack_scorefspd30, 'sb_stack_score', 'ln_fspd30_flag', 'FSPD30')
M3FSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M3FSPD30.shape}")
M3FSPD30.columns.values

The shape of dataframe after copy is:	(97, 7)


array(['start_date', 'end_date', 'sb_stack_score_FSPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [76]:
M3FSPD30.describe()

,start_date,end_date,sb_stack_score_FSPD30_gini
count,97,97,97.000000
mean,2024-02-24 00:29:41.443299072,2024-03-05 09:09:16.701030912,0.375034
min,2023-05-29 00:00:00,2023-06-04 00:00:00,0.144399
25%,2023-10-09 00:00:00,2023-10-22 00:00:00,0.326019
50%,2024-02-26 00:00:00,2024-03-03 00:00:00,0.355523
75%,2024-07-08 00:00:00,2024-07-21 00:00:00,0.425277
max,2024-11-25 00:00:00,2024-12-01 00:00:00,0.552671
std,NaN,NaN,0.077535


In [77]:
M3FSPD30.tail()

,start_date,end_date,sb_stack_score_FSPD30_gini,period,Model_Name,version,bad_rate
92,2024-11-01,2024-11-30,0.332131,Month,sb_stack_score,1.1.0,FSPD30
93,2024-11-04,2024-11-10,0.317366,Week,sb_stack_score,1.1.0,FSPD30
94,2024-11-11,2024-11-17,0.343547,Week,sb_stack_score,1.1.0,FSPD30
95,2024-11-18,2024-11-24,0.304006,Week,sb_stack_score,1.1.0,FSPD30
96,2024-11-25,2024-12-01,0.336791,Week,sb_stack_score,1.1.0,FSPD30


## FSTPD30

In [78]:
# sb_stack_score

sq = """
with sb_stack_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    sb_stack_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    sb_stack_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from sb_stack_score;
"""

df_sb_stack_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

df_sb_stack_scorefstpd30.head()

Job ID 70e47488-a610-4992-a10b-5a0018a2b7f3 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


,disbursementdate,digitalLoanAccountId,sb_stack_score,ln_fstpd30_flag,ln_mature_fstpd30_flag
0,2023-06-01 10:40:18,ac002041-b0be-446e-b173-65effe340651,0.129992,0,1
1,2023-06-01 12:56:23,e585b35d-bc13-494d-970d-2272d976991b,0.043838,0,1
2,2023-06-01 15:36:13,a3dc185d-0118-4ebc-bf03-117fdd8f9c64,0.087192,0,1
3,2023-06-01 17:17:21,3b2e2242-085e-4948-83f4-1a48bcb1b67b,0.022913,0,1
4,2023-06-01 18:40:13,ab22972c-29cf-4209-967c-3af7b19f382d,0.052938,0,1


In [79]:
gini_results = calculate_periodic_gini(df_sb_stack_scorefstpd30, 'sb_stack_score', 'ln_fstpd30_flag', 'FSTPD30')
M4FSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M4FSTPD30.shape}")
M4FSTPD30.columns.values

The shape of dataframe after copy is:	(91, 7)


array(['start_date', 'end_date', 'sb_stack_score_FSTPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [80]:
M4FSTPD30.describe()

,start_date,end_date,sb_stack_score_FSTPD30_gini
count,91,91,91.000000
mean,2024-02-06 22:09:13.846153728,2024-02-17 07:38:54.065934080,0.349580
min,2023-05-29 00:00:00,2023-06-04 00:00:00,0.162606
25%,2023-10-01 12:00:00,2023-10-11 12:00:00,0.309198
50%,2024-02-05 00:00:00,2024-02-18 00:00:00,0.339706
75%,2024-06-13 12:00:00,2024-06-26 12:00:00,0.389561
max,2024-10-21 00:00:00,2024-10-31 00:00:00,0.506416
std,NaN,NaN,0.061146


In [81]:
M4FSTPD30.tail()

,start_date,end_date,sb_stack_score_FSTPD30_gini,period,Model_Name,version,bad_rate
86,2024-09-30,2024-10-06,0.319762,Week,sb_stack_score,1.1.0,FSTPD30
87,2024-10-01,2024-10-31,0.309712,Month,sb_stack_score,1.1.0,FSTPD30
88,2024-10-07,2024-10-13,0.327533,Week,sb_stack_score,1.1.0,FSTPD30
89,2024-10-14,2024-10-20,0.297031,Week,sb_stack_score,1.1.0,FSTPD30
90,2024-10-21,2024-10-27,0.320464,Week,sb_stack_score,1.1.0,FSTPD30


## Combining the dataframes

In [82]:
import functools

dataframes = [M1FPD10, M2FPD30, M3FSPD30, M4FSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'sb_stack_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 'sb_stack_score_FPD30_gini',
       'sb_stack_score_FSPD30_gini', 'sb_stack_score_FSTPD30_gini'],
      dtype=object)

In [83]:
final_df = final_df[['start_date', 'end_date', 'period', 'Model_Name', 'version', 'bad_rate', 'sb_stack_score_FPD10_gini', 'sb_stack_score_FPD30_gini',  'sb_stack_score_FSPD30_gini', 'sb_stack_score_FSTPD30_gini']].copy()

## Creating the table 

In [84]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_sb_stack_score;"""

client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=afb850b7-0d92-4cce-8df7-83b3d5f4d520>

In [85]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('sb_stack_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('sb_stack_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('sb_stack_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('sb_stack_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_sb_stack_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=a725635b-3512-4eb9-987d-7aeb5aa28926>

# sa_stack_score

## FPD10

In [86]:
# sa_stack_score

sq = """
with sa_stack_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    sa_stack_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    sa_stack_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from sa_stack_score;
"""

sa_stack_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

sa_stack_scorefpd10.head()

Job ID 952b9a89-b679-406e-8d06-8959fc349680 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


,disbursementdate,digitalLoanAccountId,sa_stack_score,ln_fpd10_flag,ln_mature_fpd10_flag
0,2023-06-01 10:40:18,ac002041-b0be-446e-b173-65effe340651,0.084212,0,1
1,2023-06-01 12:56:23,e585b35d-bc13-494d-970d-2272d976991b,0.039108,0,1
2,2023-06-01 15:36:13,a3dc185d-0118-4ebc-bf03-117fdd8f9c64,0.054550,0,1
3,2023-06-01 17:17:21,3b2e2242-085e-4948-83f4-1a48bcb1b67b,0.025478,0,1
4,2023-06-01 18:40:13,ab22972c-29cf-4209-967c-3af7b19f382d,0.041614,0,1


In [87]:
sa_stack_scorefpd10.head()

,disbursementdate,digitalLoanAccountId,sa_stack_score,ln_fpd10_flag,ln_mature_fpd10_flag
0,2023-06-01 10:40:18,ac002041-b0be-446e-b173-65effe340651,0.084212,0,1
1,2023-06-01 12:56:23,e585b35d-bc13-494d-970d-2272d976991b,0.039108,0,1
2,2023-06-01 15:36:13,a3dc185d-0118-4ebc-bf03-117fdd8f9c64,0.054550,0,1
3,2023-06-01 17:17:21,3b2e2242-085e-4948-83f4-1a48bcb1b67b,0.025478,0,1
4,2023-06-01 18:40:13,ab22972c-29cf-4209-967c-3af7b19f382d,0.041614,0,1


In [88]:
gini_results = calculate_periodic_gini(sa_stack_scorefpd10, 'sa_stack_score', 'ln_fpd10_flag', 'FPD10')
M1FPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M1FPD10.shape}")
M1FPD10.columns.values

The shape of dataframe after copy is:	(106, 7)


array(['start_date', 'end_date', 'sa_stack_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [89]:
M1FPD10.describe()

,start_date,end_date,sa_stack_score_FPD10_gini
count,106,106,106.000000
mean,2024-03-20 14:56:36.226415104,2024-03-31 01:35:05.660377344,0.367136
min,2023-05-29 00:00:00,2023-06-04 00:00:00,0.090544
25%,2023-10-24 18:00:00,2023-11-01 06:00:00,0.324971
50%,2024-03-21 12:00:00,2024-03-31 00:00:00,0.363375
75%,2024-08-17 06:00:00,2024-08-29 12:00:00,0.409819
max,2025-01-13 00:00:00,2025-01-31 00:00:00,0.598892
std,NaN,NaN,0.084686


In [90]:
M1FPD10.head()

,start_date,end_date,sa_stack_score_FPD10_gini,period,Model_Name,version,bad_rate
0,2023-05-29,2023-06-04,0.586322,Week,sa_stack_score,1.1.0,FPD10
1,2023-06-01,2023-06-30,0.360189,Month,sa_stack_score,1.1.0,FPD10
2,2023-06-05,2023-06-11,0.456709,Week,sa_stack_score,1.1.0,FPD10
3,2023-06-12,2023-06-18,0.503145,Week,sa_stack_score,1.1.0,FPD10
4,2023-06-19,2023-06-25,0.136970,Week,sa_stack_score,1.1.0,FPD10


## FPD30

In [91]:
# sa_stack_score

sq = """
with sa_stack_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    sa_stack_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    sa_stack_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from sa_stack_score;
"""

sa_stack_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

sa_stack_scorefpd30.head()

Job ID 86ad9672-ff36-41da-9acd-575790426970 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


,disbursementdate,digitalLoanAccountId,sa_stack_score,ln_fpd30_flag,ln_mature_fpd30_flag
0,2023-06-01 10:40:18,ac002041-b0be-446e-b173-65effe340651,0.084212,0,1
1,2023-06-01 12:56:23,e585b35d-bc13-494d-970d-2272d976991b,0.039108,0,1
2,2023-06-01 15:36:13,a3dc185d-0118-4ebc-bf03-117fdd8f9c64,0.054550,0,1
3,2023-06-01 17:17:21,3b2e2242-085e-4948-83f4-1a48bcb1b67b,0.025478,0,1
4,2023-06-01 18:40:13,ab22972c-29cf-4209-967c-3af7b19f382d,0.041614,0,1


In [92]:
sa_stack_scorefpd30.head()

,disbursementdate,digitalLoanAccountId,sa_stack_score,ln_fpd30_flag,ln_mature_fpd30_flag
0,2023-06-01 10:40:18,ac002041-b0be-446e-b173-65effe340651,0.084212,0,1
1,2023-06-01 12:56:23,e585b35d-bc13-494d-970d-2272d976991b,0.039108,0,1
2,2023-06-01 15:36:13,a3dc185d-0118-4ebc-bf03-117fdd8f9c64,0.054550,0,1
3,2023-06-01 17:17:21,3b2e2242-085e-4948-83f4-1a48bcb1b67b,0.025478,0,1
4,2023-06-01 18:40:13,ab22972c-29cf-4209-967c-3af7b19f382d,0.041614,0,1


In [93]:
gini_results = calculate_periodic_gini(sa_stack_scorefpd30, 'sa_stack_score', 'ln_fpd30_flag', 'FPD30')
M2FPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M2FPD30.shape}")
M2FPD30.columns.values

The shape of dataframe after copy is:	(102, 7)


array(['start_date', 'end_date', 'sa_stack_score_FPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [94]:
M2FPD30.describe()

,start_date,end_date,sa_stack_score_FPD30_gini
count,102,102,102.000000
mean,2024-03-09 05:52:56.470588160,2024-03-19 15:03:31.764705792,0.391324
min,2023-05-29 00:00:00,2023-06-04 00:00:00,0.095902
25%,2023-10-17 18:00:00,2023-10-29 12:00:00,0.349446
50%,2024-03-07 12:00:00,2024-03-20 12:00:00,0.386761
75%,2024-07-31 06:00:00,2024-08-09 06:00:00,0.438081
max,2024-12-23 00:00:00,2024-12-31 00:00:00,0.608922
std,NaN,NaN,0.084836


In [95]:
M2FPD30.head()

,start_date,end_date,sa_stack_score_FPD30_gini,period,Model_Name,version,bad_rate
0,2023-05-29,2023-06-04,0.558182,Week,sa_stack_score,1.1.0,FPD30
1,2023-06-01,2023-06-30,0.347656,Month,sa_stack_score,1.1.0,FPD30
2,2023-06-05,2023-06-11,0.394815,Week,sa_stack_score,1.1.0,FPD30
3,2023-06-12,2023-06-18,0.485275,Week,sa_stack_score,1.1.0,FPD30
4,2023-06-19,2023-06-25,0.153949,Week,sa_stack_score,1.1.0,FPD30


## FSTPD30

In [96]:
# sa_stack_score

sq = """
with sa_stack_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    sa_stack_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    sa_stack_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from sa_stack_score;
"""

sa_stack_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

sa_stack_scorefspd30.head()

Job ID e245adbe-792c-44e7-912e-21f4058b1704 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


,disbursementdate,digitalLoanAccountId,sa_stack_score,ln_fspd30_flag,ln_mature_fspd30_flag
0,2023-06-01 10:40:18,ac002041-b0be-446e-b173-65effe340651,0.084212,0,1
1,2023-06-01 12:56:23,e585b35d-bc13-494d-970d-2272d976991b,0.039108,0,1
2,2023-06-01 15:36:13,a3dc185d-0118-4ebc-bf03-117fdd8f9c64,0.054550,0,1
3,2023-06-01 17:17:21,3b2e2242-085e-4948-83f4-1a48bcb1b67b,0.025478,0,1
4,2023-06-01 18:40:13,ab22972c-29cf-4209-967c-3af7b19f382d,0.041614,0,1


In [97]:
gini_results = calculate_periodic_gini(sa_stack_scorefspd30, 'sa_stack_score', 'ln_fspd30_flag', 'FSPD30')
M3FSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M3FSPD30.shape}")
M3FSPD30.columns.values

The shape of dataframe after copy is:	(97, 7)


array(['start_date', 'end_date', 'sa_stack_score_FSPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [98]:
M3FSPD30.tail()

,start_date,end_date,sa_stack_score_FSPD30_gini,period,Model_Name,version,bad_rate
92,2024-11-01,2024-11-30,0.359399,Month,sa_stack_score,1.1.0,FSPD30
93,2024-11-04,2024-11-10,0.357749,Week,sa_stack_score,1.1.0,FSPD30
94,2024-11-11,2024-11-17,0.357843,Week,sa_stack_score,1.1.0,FSPD30
95,2024-11-18,2024-11-24,0.348067,Week,sa_stack_score,1.1.0,FSPD30
96,2024-11-25,2024-12-01,0.275778,Week,sa_stack_score,1.1.0,FSPD30


## FSTPD30

In [99]:
# sa_stack_score

sq = """
with sa_stack_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    sa_stack_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    sa_stack_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from sa_stack_score;
"""

sa_stack_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

sa_stack_scorefstpd30.head()

Job ID 15b7c930-096a-4b08-acd8-54f9a4cc4e85 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


,disbursementdate,digitalLoanAccountId,sa_stack_score,ln_fstpd30_flag,ln_mature_fstpd30_flag
0,2023-06-01 10:40:18,ac002041-b0be-446e-b173-65effe340651,0.084212,0,1
1,2023-06-01 12:56:23,e585b35d-bc13-494d-970d-2272d976991b,0.039108,0,1
2,2023-06-01 15:36:13,a3dc185d-0118-4ebc-bf03-117fdd8f9c64,0.054550,0,1
3,2023-06-01 17:17:21,3b2e2242-085e-4948-83f4-1a48bcb1b67b,0.025478,0,1
4,2023-06-01 18:40:13,ab22972c-29cf-4209-967c-3af7b19f382d,0.041614,0,1


In [100]:
sa_stack_scorefstpd30.tail()

,disbursementdate,digitalLoanAccountId,sa_stack_score,ln_fstpd30_flag,ln_mature_fstpd30_flag
124268,2024-10-23 12:32:11,8cb3c48f-8099-4162-a3e8-9012b3027147,0.180649,1,1
124269,2023-09-19 17:16:03,73647349-7c64-4f98-9b43-9a11a9b055ab,0.058285,1,1
124270,2023-12-29 16:57:11,723f7a19-2dda-4723-9f59-05994619222c,0.075160,1,1
124271,2023-10-15 17:22:11,3089ebda-4334-483c-8ba4-d25c6d5ff451,0.096091,1,1
124272,2024-10-06 16:04:42,48ebafc1-c34c-4b56-a287-834309cb5b0f,0.139558,1,1


In [101]:
gini_results = calculate_periodic_gini(sa_stack_scorefstpd30, 'sa_stack_score', 'ln_fstpd30_flag', 'FSTPD30')
M4FSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M4FSTPD30.shape}")
M4FSTPD30.columns.values

The shape of dataframe after copy is:	(91, 7)


array(['start_date', 'end_date', 'sa_stack_score_FSTPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

## Combining the dataframes

In [102]:
import functools

dataframes = [M1FPD10, M2FPD30, M3FSPD30, M4FSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'sa_stack_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 'sa_stack_score_FPD30_gini',
       'sa_stack_score_FSPD30_gini', 'sa_stack_score_FSTPD30_gini'],
      dtype=object)

In [103]:
final_df = final_df[['start_date', 'end_date', 'period', 'Model_Name', 'version', 'bad_rate', 'sa_stack_score_FPD10_gini', 'sa_stack_score_FPD30_gini',  'sa_stack_score_FSPD30_gini', 'sa_stack_score_FSTPD30_gini']].copy()

## Creating the table 

In [104]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_sa_stack_score;"""

client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=96e7485e-d230-4b88-a613-95b081d092e1>

In [105]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('sa_stack_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('sa_stack_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('sa_stack_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('sa_stack_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_sa_stack_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=2df2b648-3ed2-466c-8298-680640c188cf>

# gen_credo_score

## FPD10

In [106]:


sq = """
with gen_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    gen_credo_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    gen_credo_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from gen_credo_score;
"""

gen_credo_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

gen_credo_scorefpd10.head()

Job ID 6259c304-bb79-4283-ac11-d53d88a70445 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


,disbursementdate,digitalLoanAccountId,gen_credo_score,ln_fpd10_flag,ln_mature_fpd10_flag
0,2024-12-30 19:14:03,64af53eb-a54c-43a4-bf62-15921a94a7cc,0.046081,0,1
1,2024-12-22 12:23:28,64ae039c-0cbe-47c0-9dec-5133167fa4fe,0.040657,0,1
2,2024-12-14 11:40:28,718b7f93-8bdf-47cd-8866-6b6768677989,0.036719,0,1
3,2024-12-21 17:00:39,5911a7ab-9836-466f-a6f5-e6561195358f,0.053590,0,1
4,2024-12-06 12:32:01,683da554-bdd7-4657-bec6-6771f9ecaae6,0.122813,0,1


In [107]:
gini_results = calculate_periodic_gini(gen_credo_scorefpd10, 'gen_credo_score', 'ln_fpd10_flag', 'FPD10')
M1FPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M1FPD10.shape}")
M1FPD10.columns.values

The shape of dataframe after copy is:	(132, 7)


array(['start_date', 'end_date', 'gen_credo_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [108]:
M1FPD10.head()

,start_date,end_date,gen_credo_score_FPD10_gini,period,Model_Name,version,bad_rate
0,2023-01-01,2023-01-31,0.246725,Month,gen_credo_score,1.1.0,FPD10
1,2023-01-02,2023-01-08,0.301125,Week,gen_credo_score,1.1.0,FPD10
2,2023-01-09,2023-01-15,0.273400,Week,gen_credo_score,1.1.0,FPD10
3,2023-01-16,2023-01-22,0.667532,Week,gen_credo_score,1.1.0,FPD10
4,2023-01-23,2023-01-29,0.086505,Week,gen_credo_score,1.1.0,FPD10


## FPD30

In [109]:
sq = """
with gen_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    gen_credo_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    gen_credo_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from gen_credo_score;
"""

gen_credo_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

gen_credo_scorefpd30.head()

Job ID 6d462f39-fc6b-4eee-8387-7a6cd8b84bf8 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


,disbursementdate,digitalLoanAccountId,gen_credo_score,ln_fpd30_flag,ln_mature_fpd30_flag
0,2023-01-02 10:19:54,73902306-ce07-4625-9d1e-a611eeb83166,0.175299,0,1
1,2023-01-02 11:39:36,d438dd4f-4c6c-44b5-ba83-58585ba8530a,0.150079,0,1
2,2023-01-02 12:04:44,44247f51-ac7a-4035-8d34-af7af2d9548c,0.099274,1,1
3,2023-01-02 12:13:00,5e85ce9b-fb16-48a4-9f62-c543782b54e1,0.152459,0,1
4,2023-01-02 12:53:05,6c53ccd6-5de3-4b2a-8216-caa1b1146cd6,0.100128,1,1


In [110]:
gen_credo_scorefpd30.head()

,disbursementdate,digitalLoanAccountId,gen_credo_score,ln_fpd30_flag,ln_mature_fpd30_flag
0,2023-01-02 10:19:54,73902306-ce07-4625-9d1e-a611eeb83166,0.175299,0,1
1,2023-01-02 11:39:36,d438dd4f-4c6c-44b5-ba83-58585ba8530a,0.150079,0,1
2,2023-01-02 12:04:44,44247f51-ac7a-4035-8d34-af7af2d9548c,0.099274,1,1
3,2023-01-02 12:13:00,5e85ce9b-fb16-48a4-9f62-c543782b54e1,0.152459,0,1
4,2023-01-02 12:53:05,6c53ccd6-5de3-4b2a-8216-caa1b1146cd6,0.100128,1,1


In [111]:
gini_results = calculate_periodic_gini(gen_credo_scorefpd30, 'gen_credo_score', 'ln_fpd30_flag', 'FPD30')
M2FPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M2FPD30.shape}")
M2FPD30.columns.values

The shape of dataframe after copy is:	(128, 7)


array(['start_date', 'end_date', 'gen_credo_score_FPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [112]:
M2FPD30.head()

,start_date,end_date,gen_credo_score_FPD30_gini,period,Model_Name,version,bad_rate
0,2023-01-01,2023-01-31,0.253005,Month,gen_credo_score,1.1.0,FPD30
1,2023-01-02,2023-01-08,0.342149,Week,gen_credo_score,1.1.0,FPD30
2,2023-01-09,2023-01-15,0.407163,Week,gen_credo_score,1.1.0,FPD30
3,2023-01-16,2023-01-22,0.667532,Week,gen_credo_score,1.1.0,FPD30
4,2023-01-23,2023-01-29,0.025547,Week,gen_credo_score,1.1.0,FPD30


## FSPD30

In [113]:
sq = """
with gen_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    gen_credo_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    gen_credo_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from gen_credo_score;
"""

gen_credo_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(gen_credo_scorefspd30.head(2))

gini_results = calculate_periodic_gini(gen_credo_scorefspd30, 'gen_credo_score', 'ln_fspd30_flag', 'FSPD30')
M3FSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M3FSPD30.shape}")
print(M3FSPD30.columns.values)

M3FSPD30.head()

Job ID 2d66e106-cfe7-4e49-9646-77fe72cee8df successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  gen_credo_score  \
0 2023-01-02 10:19:54  73902306-ce07-4625-9d1e-a611eeb83166         0.175299   
1 2023-01-02 11:39:36  d438dd4f-4c6c-44b5-ba83-58585ba8530a         0.150079   

   ln_fspd30_flag  ln_mature_fspd30_flag  
0               0                      1  
1               0                      1  
The shape of dataframe after copy is:	(123, 7)
['start_date' 'end_date' 'gen_credo_score_FSPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']


,start_date,end_date,gen_credo_score_FSPD30_gini,period,Model_Name,version,bad_rate
0,2023-01-01,2023-01-31,0.172055,Month,gen_credo_score,1.1.0,FSPD30
1,2023-01-02,2023-01-08,0.262803,Week,gen_credo_score,1.1.0,FSPD30
2,2023-01-09,2023-01-15,0.189744,Week,gen_credo_score,1.1.0,FSPD30
3,2023-01-16,2023-01-22,0.410738,Week,gen_credo_score,1.1.0,FSPD30
4,2023-01-23,2023-01-29,0.073579,Week,gen_credo_score,1.1.0,FSPD30


## FSTPD30

In [114]:
sq = """
with gen_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    gen_credo_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    gen_credo_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from gen_credo_score;
"""

gen_credo_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(gen_credo_scorefstpd30.head(2))

gini_results = calculate_periodic_gini(gen_credo_scorefstpd30, 'gen_credo_score', 'ln_fstpd30_flag', 'FSTPD30')
M4FSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M4FSTPD30.shape}")
print(M4FSTPD30.columns.values)

M4FSTPD30.head()

Job ID 54001890-6e26-420b-9d22-50c512318677 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  gen_credo_score  \
0 2023-01-02 10:19:54  73902306-ce07-4625-9d1e-a611eeb83166         0.175299   
1 2023-01-02 11:39:36  d438dd4f-4c6c-44b5-ba83-58585ba8530a         0.150079   

   ln_fstpd30_flag  ln_mature_fstpd30_flag  
0                0                       1  
1                0                       1  
The shape of dataframe after copy is:	(117, 7)
['start_date' 'end_date' 'gen_credo_score_FSTPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']


,start_date,end_date,gen_credo_score_FSTPD30_gini,period,Model_Name,version,bad_rate
0,2023-01-01,2023-01-31,0.180837,Month,gen_credo_score,1.1.0,FSTPD30
1,2023-01-02,2023-01-08,0.282959,Week,gen_credo_score,1.1.0,FSTPD30
2,2023-01-09,2023-01-15,0.189908,Week,gen_credo_score,1.1.0,FSTPD30
3,2023-01-16,2023-01-22,0.198840,Week,gen_credo_score,1.1.0,FSTPD30
4,2023-01-23,2023-01-29,0.162845,Week,gen_credo_score,1.1.0,FSTPD30


## Combining tables 

In [115]:
import functools

dataframes = [M1FPD10, M2FPD30, M3FSPD30, M4FSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'gen_credo_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 'gen_credo_score_FPD30_gini',
       'gen_credo_score_FSPD30_gini', 'gen_credo_score_FSTPD30_gini'],
      dtype=object)

In [116]:
final_df = final_df[['start_date', 'end_date','period',
       'Model_Name', 'version', 'bad_rate', 'gen_credo_score_FPD10_gini','gen_credo_score_FPD30_gini',   'gen_credo_score_FSPD30_gini', 'gen_credo_score_FSTPD30_gini']].copy()

In [117]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_gen_credo_score;"""

client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=6cc72c2d-0d33-4669-a090-8452f3674487>

In [118]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('gen_credo_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('gen_credo_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('gen_credo_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('gen_credo_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_gen_credo_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=2144bc04-20e3-4b54-824b-cc2a13d7291c>

# c_credo_score

In [119]:
sq = """
with c_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    c_credo_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    c_credo_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from c_credo_score;
"""

c_credo_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

c_credo_scorefpd10.head()

gini_results = calculate_periodic_gini(c_credo_scorefpd10, 'c_credo_score', 'ln_fpd10_flag', 'FPD10')
M1FPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M1FPD10.shape}")
M1FPD10.columns.values

# FPD30

sq = """
with c_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    c_credo_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    c_credo_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from c_credo_score;
"""

c_credo_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

c_credo_scorefpd30.head()

gini_results = calculate_periodic_gini(c_credo_scorefpd30, 'c_credo_score', 'ln_fpd30_flag', 'FPD30')
M2FPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M2FPD30.shape}")
M2FPD30.columns.values

sq = """
with c_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    c_credo_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    c_credo_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from c_credo_score;
"""

c_credo_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(c_credo_scorefspd30.head(2))

gini_results = calculate_periodic_gini(c_credo_scorefspd30, 'c_credo_score', 'ln_fspd30_flag', 'FSPD30')
M3FSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M3FSPD30.shape}")
print(M3FSPD30.columns.values)

M3FSPD30.head()

sq = """
with c_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    c_credo_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    c_credo_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from c_credo_score;
"""

c_credo_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(c_credo_scorefstpd30.head(2))

gini_results = calculate_periodic_gini(c_credo_scorefstpd30, 'c_credo_score', 'ln_fstpd30_flag', 'FSTPD30')
M4FSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M4FSTPD30.shape}")
print(M4FSTPD30.columns.values)

M4FSTPD30.head()



Job ID 238d22c8-d822-4638-bf65-0670575f417c successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(132, 7)
Job ID 8e821f8d-f594-45d2-8239-c670a12802a9 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(128, 7)
Job ID 58e3f17c-077b-4fe5-8ff5-3e7611ed4943 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  c_credo_score  \
0 2023-01-02 10:19:54  73902306-ce07-4625-9d1e-a611eeb83166       0.198732   
1 2023-01-02 11:39:36  d438dd4f-4c6c-44b5-ba83-58585ba8530a       0.198170   

   ln_fspd30_flag  ln_mature_fspd30_flag  
0               0                      1  
1               0                      1  
The shape of dataframe after copy is:	(123, 7)
['start_date' 'end_date' 'c_credo_score_FSPD30_gini' 'period' 'Model_Name'
 'version' 'bad_rate']
Job ID 37d0f6d3-fe9a-4500-a125-61ea9561a6a4 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  c_credo_score  \
0 2023-01-02 10:19:54  73902306-ce07-4625-9d1e-a611eeb83166       0.198732   
1 2023-01-02 11:39:36  d438dd4f-4c6c-44b5-ba83-58585ba8530a       0.198170   

   ln_fstpd30_flag  ln_mature_fstpd30_flag  
0                0                       1  
1                0                       1  
The shape of dataframe after copy is:	(117, 7)
['start_date' 'end_date' 'c_credo_score_FSTPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']


,start_date,end_date,c_credo_score_FSTPD30_gini,period,Model_Name,version,bad_rate
0,2023-01-01,2023-01-31,0.035948,Month,c_credo_score,1.1.0,FSTPD30
1,2023-01-02,2023-01-08,0.036009,Week,c_credo_score,1.1.0,FSTPD30
2,2023-01-09,2023-01-15,0.062143,Week,c_credo_score,1.1.0,FSTPD30
3,2023-01-16,2023-01-22,0.275891,Week,c_credo_score,1.1.0,FSTPD30
4,2023-01-23,2023-01-29,-0.050558,Week,c_credo_score,1.1.0,FSTPD30


In [120]:
import functools

dataframes = [M1FPD10, M2FPD30, M3FSPD30, M4FSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'c_credo_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 'c_credo_score_FPD30_gini',
       'c_credo_score_FSPD30_gini', 'c_credo_score_FSTPD30_gini'],
      dtype=object)

In [121]:
final_df = final_df[['start_date', 'end_date','period',    'Model_Name', 'version', 'bad_rate','c_credo_score_FPD10_gini', 'c_credo_score_FPD30_gini',    'c_credo_score_FSPD30_gini', 'c_credo_score_FSTPD30_gini']].copy()

In [122]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_c_credo_score;"""
client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=4954e770-4a33-447e-8dd9-05b04b32537f>

In [123]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('c_credo_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('c_credo_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('c_credo_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('c_credo_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_c_credo_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=e9e45c3b-89cf-4074-a718-c5f3f7de51ee>

# s_credo_score

In [124]:


sq = """
with s_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    s_credo_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    s_credo_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from s_credo_score;
"""

s_credo_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

s_credo_scorefpd10.head()

gini_results = calculate_periodic_gini(s_credo_scorefpd10, 's_credo_score', 'ln_fpd10_flag', 'FPD10')
M1FPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M1FPD10.shape}")
M1FPD10.columns.values

# FPD30

sq = """
with s_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    s_credo_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    s_credo_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from s_credo_score;
"""

s_credo_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

s_credo_scorefpd30.head()

gini_results = calculate_periodic_gini(s_credo_scorefpd30, 's_credo_score', 'ln_fpd30_flag', 'FPD30')
M2FPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M2FPD30.shape}")
M2FPD30.columns.values

sq = """
with s_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    s_credo_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    s_credo_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from s_credo_score;
"""

s_credo_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(s_credo_scorefspd30.head(2))

gini_results = calculate_periodic_gini(s_credo_scorefspd30, 's_credo_score', 'ln_fspd30_flag', 'FSPD30')
M3FSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M3FSPD30.shape}")
print(M3FSPD30.columns.values)

M3FSPD30.head()

sq = """
with s_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    s_credo_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    s_credo_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from s_credo_score;
"""

s_credo_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(s_credo_scorefstpd30.head(2))

gini_results = calculate_periodic_gini(s_credo_scorefstpd30, 's_credo_score', 'ln_fstpd30_flag', 'FSTPD30')
M4FSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M4FSTPD30.shape}")
print(M4FSTPD30.columns.values)

M4FSTPD30.head()



Job ID a9f62f86-fc6b-45ef-a8a2-d6748ff5770a successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(132, 7)
Job ID 7d2667e1-c70e-462e-8605-c2fc69105b42 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(128, 7)
Job ID 442ebc68-2c0b-409d-b49d-b2f04b78b9dc successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  s_credo_score  \
0 2024-11-08 12:25:53  c686a72a-d13c-4d1d-b87e-2fe4e9e6e2ba       0.082273   
1 2024-01-02 18:57:09  8c065b9e-cb53-47a8-98f5-caf53ae90c3a       0.128757   

   ln_fspd30_flag  ln_mature_fspd30_flag  
0               1                      1  
1               0                      1  
The shape of dataframe after copy is:	(123, 7)
['start_date' 'end_date' 's_credo_score_FSPD30_gini' 'period' 'Model_Name'
 'version' 'bad_rate']
Job ID 61eb9ee4-5c09-4cdf-8545-def1ce1e0ae0 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  s_credo_score  \
0 2023-01-02 10:19:54  73902306-ce07-4625-9d1e-a611eeb83166       0.151148   
1 2023-01-02 11:39:36  d438dd4f-4c6c-44b5-ba83-58585ba8530a       0.122682   

   ln_fstpd30_flag  ln_mature_fstpd30_flag  
0                0                       1  
1                0                       1  
The shape of dataframe after copy is:	(117, 7)
['start_date' 'end_date' 's_credo_score_FSTPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']


,start_date,end_date,s_credo_score_FSTPD30_gini,period,Model_Name,version,bad_rate
0,2023-01-01,2023-01-31,0.219086,Month,s_credo_score,1.1.0,FSTPD30
1,2023-01-02,2023-01-08,0.261708,Week,s_credo_score,1.1.0,FSTPD30
2,2023-01-09,2023-01-15,0.251056,Week,s_credo_score,1.1.0,FSTPD30
3,2023-01-16,2023-01-22,0.275062,Week,s_credo_score,1.1.0,FSTPD30
4,2023-01-23,2023-01-29,0.157554,Week,s_credo_score,1.1.0,FSTPD30


## Combining data

In [125]:
import functools

dataframes = [M1FPD10, M2FPD30, M3FSPD30, M4FSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 's_credo_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 's_credo_score_FPD30_gini',
       's_credo_score_FSPD30_gini', 's_credo_score_FSTPD30_gini'],
      dtype=object)

In [126]:
final_df = final_df[['start_date', 'end_date','period',    'Model_Name', 'version', 'bad_rate','s_credo_score_FPD10_gini', 's_credo_score_FPD30_gini',    's_credo_score_FSPD30_gini', 's_credo_score_FSTPD30_gini']].copy()

In [127]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_s_credo_score;"""

client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=0e272095-fe8a-4fa2-be34-24ec8b45b0de>

In [128]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('s_credo_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('s_credo_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('s_credo_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('s_credo_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_s_credo_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=3d623b12-7cf5-4d9e-b09b-d8cc2a85768e>

In [129]:
final_df.head()

,start_date,end_date,period,Model_Name,version,bad_rate,s_credo_score_FPD10_gini,s_credo_score_FPD30_gini,s_credo_score_FSPD30_gini,s_credo_score_FSTPD30_gini
0,2023-01-01,2023-01-31,Month,s_credo_score,1.1.0,FPD10,0.230181,NaN,NaN,NaN
1,2023-01-02,2023-01-08,Week,s_credo_score,1.1.0,FPD10,0.248978,NaN,NaN,NaN
2,2023-01-09,2023-01-15,Week,s_credo_score,1.1.0,FPD10,0.296337,NaN,NaN,NaN
3,2023-01-16,2023-01-22,Week,s_credo_score,1.1.0,FPD10,0.529870,NaN,NaN,NaN
4,2023-01-23,2023-01-29,Week,s_credo_score,1.1.0,FPD10,0.079585,NaN,NaN,NaN


# fu_credo_score

In [130]:


sq = """
with fu_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    fu_credo_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    fu_credo_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from fu_credo_score;
"""

fu_credo_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

fu_credo_scorefpd10.head()

gini_results = calculate_periodic_gini(fu_credo_scorefpd10, 'fu_credo_score', 'ln_fpd10_flag', 'FPD10')
M1FPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M1FPD10.shape}")
M1FPD10.columns.values

# FPD30

sq = """
with fu_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    fu_credo_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    fu_credo_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from fu_credo_score;
"""

fu_credo_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

fu_credo_scorefpd30.head()

gini_results = calculate_periodic_gini(fu_credo_scorefpd30, 'fu_credo_score', 'ln_fpd30_flag', 'FPD30')
M2FPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M2FPD30.shape}")
M2FPD30.columns.values

sq = """
with fu_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    fu_credo_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    fu_credo_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from fu_credo_score;
"""

fu_credo_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(fu_credo_scorefspd30.head(2))

gini_results = calculate_periodic_gini(fu_credo_scorefspd30, 'fu_credo_score', 'ln_fspd30_flag', 'FSPD30')
M3FSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M3FSPD30.shape}")
print(M3FSPD30.columns.values)

M3FSPD30.head()

sq = """
with fu_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    fu_credo_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    fu_credo_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from fu_credo_score;
"""

fu_credo_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(fu_credo_scorefstpd30.head(2))

gini_results = calculate_periodic_gini(fu_credo_scorefstpd30, 'fu_credo_score', 'ln_fstpd30_flag', 'FSTPD30')
M4FSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M4FSTPD30.shape}")
print(M4FSTPD30.columns.values)

M4FSTPD30.head()



Job ID f9f3d695-ca17-47f7-b5cb-96707a014c6f successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(132, 7)
Job ID d462e15f-5f84-4b2c-8ec2-f76263836d39 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(128, 7)
Job ID 1a56e411-eb17-40a0-b97b-75da9f04a8db successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  fu_credo_score  \
0 2023-01-02 10:19:54  73902306-ce07-4625-9d1e-a611eeb83166        0.096835   
1 2023-01-02 11:39:36  d438dd4f-4c6c-44b5-ba83-58585ba8530a        0.198170   

   ln_fspd30_flag  ln_mature_fspd30_flag  
0               0                      1  
1               0                      1  
The shape of dataframe after copy is:	(123, 7)
['start_date' 'end_date' 'fu_credo_score_FSPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']
Job ID 4bad4ea7-f953-4665-9eb5-0fb78b66ed41 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  fu_credo_score  \
0 2023-01-02 10:19:54  73902306-ce07-4625-9d1e-a611eeb83166        0.096835   
1 2023-01-02 11:39:36  d438dd4f-4c6c-44b5-ba83-58585ba8530a        0.198170   

   ln_fstpd30_flag  ln_mature_fstpd30_flag  
0                0                       1  
1                0                       1  
The shape of dataframe after copy is:	(117, 7)
['start_date' 'end_date' 'fu_credo_score_FSTPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']


,start_date,end_date,fu_credo_score_FSTPD30_gini,period,Model_Name,version,bad_rate
0,2023-01-01,2023-01-31,0.072860,Month,fu_credo_score,1.1.0,FSTPD30
1,2023-01-02,2023-01-08,0.002165,Week,fu_credo_score,1.1.0,FSTPD30
2,2023-01-09,2023-01-15,0.056674,Week,fu_credo_score,1.1.0,FSTPD30
3,2023-01-16,2023-01-22,0.326429,Week,fu_credo_score,1.1.0,FSTPD30
4,2023-01-23,2023-01-29,0.109935,Week,fu_credo_score,1.1.0,FSTPD30


## Combining data

In [131]:
import functools

dataframes = [M1FPD10, M2FPD30, M3FSPD30, M4FSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'fu_credo_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 'fu_credo_score_FPD30_gini',
       'fu_credo_score_FSPD30_gini', 'fu_credo_score_FSTPD30_gini'],
      dtype=object)

In [132]:
final_df = final_df[['start_date', 'end_date','period',    'Model_Name', 'version', 'bad_rate','fu_credo_score_FPD10_gini', 'fu_credo_score_FPD30_gini',    'fu_credo_score_FSPD30_gini', 'fu_credo_score_FSTPD30_gini']].copy()

In [133]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_fu_credo_score"""
client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=e37509dd-d513-4a43-8d3f-41bd9a6a25eb>

In [134]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('fu_credo_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('fu_credo_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('fu_credo_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('fu_credo_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_fu_credo_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=e11ddfa8-01af-404c-85f6-cd20b30aed0c>

In [135]:
final_df.head()

,start_date,end_date,period,Model_Name,version,bad_rate,fu_credo_score_FPD10_gini,fu_credo_score_FPD30_gini,fu_credo_score_FSPD30_gini,fu_credo_score_FSTPD30_gini
0,2023-01-01,2023-01-31,Month,fu_credo_score,1.1.0,FPD10,0.023608,NaN,NaN,NaN
1,2023-01-02,2023-01-08,Week,fu_credo_score,1.1.0,FPD10,-0.032464,NaN,NaN,NaN
2,2023-01-09,2023-01-15,Week,fu_credo_score,1.1.0,FPD10,0.016278,NaN,NaN,NaN
3,2023-01-16,2023-01-22,Week,fu_credo_score,1.1.0,FPD10,0.153247,NaN,NaN,NaN
4,2023-01-23,2023-01-29,Week,fu_credo_score,1.1.0,FPD10,0.120242,NaN,NaN,NaN


# r_credo_score

In [136]:


sq = """
with r_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    r_credo_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    r_credo_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from r_credo_score;
"""

r_credo_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

r_credo_scorefpd10.head()

gini_results = calculate_periodic_gini(r_credo_scorefpd10, 'r_credo_score', 'ln_fpd10_flag', 'FPD10')
M1FPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M1FPD10.shape}")
M1FPD10.columns.values

# FPD30

sq = """
with r_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    r_credo_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    r_credo_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from r_credo_score;
"""

r_credo_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

r_credo_scorefpd30.head()

gini_results = calculate_periodic_gini(r_credo_scorefpd30, 'r_credo_score', 'ln_fpd30_flag', 'FPD30')
M2FPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M2FPD30.shape}")
M2FPD30.columns.values

sq = """
with r_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    r_credo_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    r_credo_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from r_credo_score;
"""

r_credo_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(r_credo_scorefspd30.head(2))

gini_results = calculate_periodic_gini(r_credo_scorefspd30, 'r_credo_score', 'ln_fspd30_flag', 'FSPD30')
M3FSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M3FSPD30.shape}")
print(M3FSPD30.columns.values)

M3FSPD30.head()

sq = """
with r_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    r_credo_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    r_credo_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from r_credo_score;
"""

r_credo_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(r_credo_scorefstpd30.head(2))

gini_results = calculate_periodic_gini(r_credo_scorefstpd30, 'r_credo_score', 'ln_fstpd30_flag', 'FSTPD30')
M4FSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M4FSTPD30.shape}")
print(M4FSTPD30.columns.values)

M4FSTPD30.head()



Job ID ff03cf91-2997-48f8-a2f7-778f7ae5a4bb successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(132, 7)
Job ID eb158dc7-3174-4d7d-adf3-2764ae256e98 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(128, 7)
Job ID 47fb6482-deec-43e7-a850-2685754db101 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  r_credo_score  \
0 2023-01-02 10:19:54  73902306-ce07-4625-9d1e-a611eeb83166       0.276277   
1 2023-01-02 11:39:36  d438dd4f-4c6c-44b5-ba83-58585ba8530a       0.150079   

   ln_fspd30_flag  ln_mature_fspd30_flag  
0               0                      1  
1               0                      1  
The shape of dataframe after copy is:	(123, 7)
['start_date' 'end_date' 'r_credo_score_FSPD30_gini' 'period' 'Model_Name'
 'version' 'bad_rate']
Job ID ed8dbf4c-b34d-4af8-b931-fe1e8607d3cc successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  r_credo_score  \
0 2024-01-02 18:57:09  8c065b9e-cb53-47a8-98f5-caf53ae90c3a       0.168724   
1 2023-12-17 11:58:35  b6475dcf-de0e-4576-b2b1-008120d4942c       0.134869   

   ln_fstpd30_flag  ln_mature_fstpd30_flag  
0                0                       1  
1                0                       1  
The shape of dataframe after copy is:	(117, 7)
['start_date' 'end_date' 'r_credo_score_FSTPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']


,start_date,end_date,r_credo_score_FSTPD30_gini,period,Model_Name,version,bad_rate
0,2023-01-01,2023-01-31,0.048984,Month,r_credo_score,1.1.0,FSTPD30
1,2023-01-02,2023-01-08,0.014955,Week,r_credo_score,1.1.0,FSTPD30
2,2023-01-09,2023-01-15,0.193885,Week,r_credo_score,1.1.0,FSTPD30
3,2023-01-16,2023-01-22,-0.019056,Week,r_credo_score,1.1.0,FSTPD30
4,2023-01-23,2023-01-29,0.024691,Week,r_credo_score,1.1.0,FSTPD30


## Combining data

In [137]:
import functools

dataframes = [M1FPD10, M2FPD30, M3FSPD30, M4FSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'r_credo_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 'r_credo_score_FPD30_gini',
       'r_credo_score_FSPD30_gini', 'r_credo_score_FSTPD30_gini'],
      dtype=object)

In [138]:
final_df = final_df[['start_date', 'end_date', 'period',   'Model_Name', 'version', 'bad_rate','r_credo_score_FPD10_gini','r_credo_score_FPD30_gini',    'r_credo_score_FSPD30_gini', 'r_credo_score_FSTPD30_gini']].copy()

In [139]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_r_credo_score;"""
client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=2be76c29-5a60-44d8-b602-92542d2b5257>

In [140]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('r_credo_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('r_credo_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('r_credo_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('r_credo_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_r_credo_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=7052dbb6-8c9d-4034-a33d-17bcfc32f7a1>

# old_gen_credo_score

In [141]:


sq = """
with old_gen_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    old_gen_credo_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    old_gen_credo_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from old_gen_credo_score;
"""

old_gen_credo_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

old_gen_credo_scorefpd10.head()

gini_results = calculate_periodic_gini_threedigit(old_gen_credo_scorefpd10, 'old_gen_credo_score', 'ln_fpd10_flag', 'FPD10')
M1FPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M1FPD10.shape}")
M1FPD10.columns.values

# FPD30

sq = """
with old_gen_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    old_gen_credo_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    old_gen_credo_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from old_gen_credo_score;
"""

old_gen_credo_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

old_gen_credo_scorefpd30.head()

gini_results = calculate_periodic_gini_threedigit(old_gen_credo_scorefpd30, 'old_gen_credo_score', 'ln_fpd30_flag', 'FPD30')
M2FPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M2FPD30.shape}")
M2FPD30.columns.values

sq = """
with old_gen_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    old_gen_credo_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    old_gen_credo_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from old_gen_credo_score;
"""

old_gen_credo_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(old_gen_credo_scorefspd30.head(2))

gini_results = calculate_periodic_gini_threedigit(old_gen_credo_scorefspd30, 'old_gen_credo_score', 'ln_fspd30_flag', 'FSPD30')
M3FSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M3FSPD30.shape}")
print(M3FSPD30.columns.values)

M3FSPD30.head()

sq = """
with old_gen_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    old_gen_credo_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    old_gen_credo_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from old_gen_credo_score;
"""

old_gen_credo_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(old_gen_credo_scorefstpd30.head(2))

gini_results = calculate_periodic_gini_threedigit(old_gen_credo_scorefstpd30, 'old_gen_credo_score', 'ln_fstpd30_flag', 'FSTPD30')
M4FSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M4FSTPD30.shape}")
print(M4FSTPD30.columns.values)

M4FSTPD30.head()



Job ID fa5ba422-8315-4858-8630-cb171b7b4a9f successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(132, 7)
Job ID 30cd7376-5181-4a68-bba1-d000e65de7d6 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(128, 7)
Job ID f34172c3-3188-4fea-a2f6-f5b4a020b561 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  \
0 2023-01-02 12:04:44  44247f51-ac7a-4035-8d34-af7af2d9548c   
1 2023-01-02 12:53:05  6c53ccd6-5de3-4b2a-8216-caa1b1146cd6   

  old_gen_credo_score  ln_fspd30_flag  ln_mature_fspd30_flag  
0               498.0               1                      1  
1               481.0               1                      1  
The shape of dataframe after copy is:	(123, 7)
['start_date' 'end_date' 'old_gen_credo_score_FSPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']
Job ID 4c0c4983-60a8-48e2-9e02-6892b355b70e successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  \
0 2023-01-02 12:04:44  44247f51-ac7a-4035-8d34-af7af2d9548c   
1 2023-01-02 12:53:05  6c53ccd6-5de3-4b2a-8216-caa1b1146cd6   

  old_gen_credo_score  ln_fstpd30_flag  ln_mature_fstpd30_flag  
0               498.0                1                       1  
1               481.0                1                       1  
The shape of dataframe after copy is:	(117, 7)
['start_date' 'end_date' 'old_gen_credo_score_FSTPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']


,start_date,end_date,old_gen_credo_score_FSTPD30_gini,period,Model_Name,version,bad_rate
0,2023-01-01,2023-01-31,1.223497e-01,Month,old_gen_credo_score,1.1.0,FSTPD30
1,2023-01-02,2023-01-08,2.303313e-01,Week,old_gen_credo_score,1.1.0,FSTPD30
2,2023-01-09,2023-01-15,1.427644e-01,Week,old_gen_credo_score,1.1.0,FSTPD30
3,2023-01-16,2023-01-22,1.336406e-01,Week,old_gen_credo_score,1.1.0,FSTPD30
4,2023-01-23,2023-01-29,-2.220446e-16,Week,old_gen_credo_score,1.1.0,FSTPD30


## Combining data

In [142]:
import functools

dataframes = [M1FPD10, M2FPD30, M3FSPD30, M4FSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'old_gen_credo_score_FPD10_gini',
       'period', 'Model_Name', 'version', 'bad_rate',
       'old_gen_credo_score_FPD30_gini',
       'old_gen_credo_score_FSPD30_gini',
       'old_gen_credo_score_FSTPD30_gini'], dtype=object)

In [143]:
final_df = final_df[['start_date', 'end_date', 'period',   'Model_Name', 'version', 'bad_rate','old_gen_credo_score_FPD10_gini','old_gen_credo_score_FPD30_gini',    'old_gen_credo_score_FSPD30_gini', 'old_gen_credo_score_FSTPD30_gini']].copy()

In [144]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_old_gen_credo_score;"""
client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=5e7ab9c1-f0dc-43a4-88e3-eacdb9651744>

In [145]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('old_gen_credo_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('old_gen_credo_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('old_gen_credo_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('old_gen_credo_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_old_gen_credo_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=67c43c9d-07d0-4d2e-b3d2-443c59ed5fc7>

In [146]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_old_gen_credo_score;"""

df = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID 6681e0a0-1f59-403f-b595-6be7a44bb4a6 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [147]:
df['bad_rate'].value_counts()

bad_rate
FPD10      132
FPD30      128
FSPD30     123
FSTPD30    117
Name: count, dtype: int64

# old_cic_score

In [148]:


sq = """
with old_cic_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    old_cic_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    old_cic_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from old_cic_score;
"""

old_cic_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

old_cic_scorefpd10.head()

gini_results = calculate_periodic_hybrid_gini(old_cic_scorefpd10, 'old_cic_score', 'ln_fpd10_flag', 'FPD10')
M1FPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M1FPD10.shape}")
M1FPD10.columns.values

# FPD30

sq = """
with old_cic_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    old_cic_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    old_cic_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from old_cic_score;
"""

old_cic_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

old_cic_scorefpd30.head()

gini_results = calculate_periodic_hybrid_gini(old_cic_scorefpd30, 'old_cic_score', 'ln_fpd30_flag', 'FPD30')
M2FPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M2FPD30.shape}")
M2FPD30.columns.values

sq = """
with old_cic_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    old_cic_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    old_cic_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from old_cic_score;
"""

old_cic_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(old_cic_scorefspd30.head(2))

gini_results = calculate_periodic_hybrid_gini(old_cic_scorefspd30, 'old_cic_score', 'ln_fspd30_flag', 'FSPD30')
M3FSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M3FSPD30.shape}")
print(M3FSPD30.columns.values)

M3FSPD30.head()

sq = """
with old_cic_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    old_cic_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    old_cic_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from old_cic_score;
"""

old_cic_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(old_cic_scorefstpd30.head(2))

gini_results = calculate_periodic_hybrid_gini(old_cic_scorefstpd30, 'old_cic_score', 'ln_fstpd30_flag', 'FSTPD30')
M4FSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M4FSTPD30.shape}")
print(M4FSTPD30.columns.values)

M4FSTPD30.head()



Job ID 54d862ff-df07-49a0-989e-e8bbb40d3333 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(92, 7)
Job ID 3bdeadf7-a696-487e-8e9b-07fb58ed25db successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(88, 7)
Job ID 0b11f384-11e6-4939-a76c-40673c916266 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId old_cic_score  \
0 2023-08-23 15:36:50  c8e6977b-a5ca-48b0-9ef0-e6e9581a7229       484.000   
1 2023-08-23 20:04:02  9c7f4086-33c8-4b8f-b1b1-f95cc0b104b2       480.000   

   ln_fspd30_flag  ln_mature_fspd30_flag  
0               0                      1  
1               1                      1  
The shape of dataframe after copy is:	(83, 7)
['start_date' 'end_date' 'old_cic_score_FSPD30_gini' 'period' 'Model_Name'
 'version' 'bad_rate']
Job ID d8fd68f7-a673-4cad-9afd-c9aea68e7c2d successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId old_cic_score  \
0 2023-08-23 15:36:50  c8e6977b-a5ca-48b0-9ef0-e6e9581a7229       484.000   
1 2023-08-23 20:04:02  9c7f4086-33c8-4b8f-b1b1-f95cc0b104b2       480.000   

   ln_fstpd30_flag  ln_mature_fstpd30_flag  
0                1                       1  
1                1                       1  
The shape of dataframe after copy is:	(77, 7)
['start_date' 'end_date' 'old_cic_score_FSTPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']


,start_date,end_date,old_cic_score_FSTPD30_gini,period,Model_Name,version,bad_rate
0,2023-08-01,2023-08-31,0.073366,Month,old_cic_score,1.1.0,FSTPD30
1,2023-08-21,2023-08-27,0.046897,Week,old_cic_score,1.1.0,FSTPD30
2,2023-08-28,2023-09-03,0.132275,Week,old_cic_score,1.1.0,FSTPD30
3,2023-09-01,2023-09-30,0.142841,Month,old_cic_score,1.1.0,FSTPD30
4,2023-09-04,2023-09-10,0.159933,Week,old_cic_score,1.1.0,FSTPD30


## Combining data

In [149]:
import functools

dataframes = [M1FPD10, M2FPD30, M3FSPD30, M4FSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'old_cic_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 'old_cic_score_FPD30_gini',
       'old_cic_score_FSPD30_gini', 'old_cic_score_FSTPD30_gini'],
      dtype=object)

In [150]:
final_df = final_df[['start_date', 'end_date', 'period',   'Model_Name', 'version', 'bad_rate','old_cic_score_FPD10_gini','old_cic_score_FPD30_gini',    'old_cic_score_FSPD30_gini', 'old_cic_score_FSTPD30_gini']].copy()

In [151]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_old_cic_score;"""
client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=a44e3a3e-6946-4d05-a6a7-a5f169e9536e>

In [152]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('old_cic_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('old_cic_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('old_cic_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('old_cic_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_old_cic_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=e3e12156-a240-4aaa-ad94-c7c4c977d1eb>

# old_demo_score

In [153]:


sq = """
with old_demo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    old_demo_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    old_demo_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from old_demo_score;
"""

old_demo_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

old_demo_scorefpd10.head()

gini_results = calculate_periodic_hybrid_gini(old_demo_scorefpd10, 'old_demo_score', 'ln_fpd10_flag', 'FPD10')
M1FPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M1FPD10.shape}")
M1FPD10.columns.values

# FPD30

sq = """
with old_demo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    old_demo_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    old_demo_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from old_demo_score;
"""

old_demo_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

old_demo_scorefpd30.head()

gini_results = calculate_periodic_hybrid_gini(old_demo_scorefpd30, 'old_demo_score', 'ln_fpd30_flag', 'FPD30')
M2FPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M2FPD30.shape}")
M2FPD30.columns.values

sq = """
with old_demo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    old_demo_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    old_demo_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from old_demo_score;
"""

old_demo_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(old_demo_scorefspd30.head(2))

gini_results = calculate_periodic_hybrid_gini(old_demo_scorefspd30, 'old_demo_score', 'ln_fspd30_flag', 'FSPD30')
M3FSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M3FSPD30.shape}")
print(M3FSPD30.columns.values)

M3FSPD30.head()

sq = """
with old_demo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    old_demo_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    old_demo_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from old_demo_score;
"""

old_demo_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(old_demo_scorefstpd30.head(2))

gini_results = calculate_periodic_hybrid_gini(old_demo_scorefstpd30, 'old_demo_score', 'ln_fstpd30_flag', 'FSTPD30')
M4FSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M4FSTPD30.shape}")
print(M4FSTPD30.columns.values)

M4FSTPD30.head()



Job ID e3bcd1ae-4b29-4313-a5d5-eb12a9149403 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(132, 7)
Job ID 545bdb59-cf9b-472d-9cde-80368a55bf3d successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(128, 7)
Job ID 2a045272-ec8e-4e4a-9179-7ac8e681e056 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  old_demo_score  \
0 2024-11-08 12:25:53  c686a72a-d13c-4d1d-b87e-2fe4e9e6e2ba           424.0   
1 2023-12-17 11:58:35  b6475dcf-de0e-4576-b2b1-008120d4942c           502.0   

   ln_fspd30_flag  ln_mature_fspd30_flag  
0               1                      1  
1               0                      1  
The shape of dataframe after copy is:	(123, 7)
['start_date' 'end_date' 'old_demo_score_FSPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']
Job ID 5a7d74f2-7c30-422e-820b-e063d7b85f0f successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  old_demo_score  \
0 2023-12-17 11:58:35  b6475dcf-de0e-4576-b2b1-008120d4942c           502.0   
1 2023-06-22 15:10:45  41d9564d-dbec-4eeb-acc1-21412cbc5b1b           504.0   

   ln_fstpd30_flag  ln_mature_fstpd30_flag  
0                0                       1  
1                0                       1  
The shape of dataframe after copy is:	(117, 7)
['start_date' 'end_date' 'old_demo_score_FSTPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']


,start_date,end_date,old_demo_score_FSTPD30_gini,period,Model_Name,version,bad_rate
0,2023-01-01,2023-01-31,0.122498,Month,old_demo_score,1.1.0,FSTPD30
1,2023-01-02,2023-01-08,0.177489,Week,old_demo_score,1.1.0,FSTPD30
2,2023-01-09,2023-01-15,0.037534,Week,old_demo_score,1.1.0,FSTPD30
3,2023-01-16,2023-01-22,0.148302,Week,old_demo_score,1.1.0,FSTPD30
4,2023-01-23,2023-01-29,0.128748,Week,old_demo_score,1.1.0,FSTPD30


## Combining data

In [154]:
import functools

dataframes = [M1FPD10, M2FPD30, M3FSPD30, M4FSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'old_demo_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 'old_demo_score_FPD30_gini',
       'old_demo_score_FSPD30_gini', 'old_demo_score_FSTPD30_gini'],
      dtype=object)

In [155]:
final_df = final_df[['start_date', 'end_date', 'period',   'Model_Name', 'version', 'bad_rate','old_demo_score_FPD10_gini','old_demo_score_FPD30_gini',    'old_demo_score_FSPD30_gini', 'old_demo_score_FSTPD30_gini']].copy()

In [156]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_old_demo_score;"""
client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=b6dedb74-f3e4-468c-8f9f-c3c8e1fa97f6>

In [157]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('old_demo_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('old_demo_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('old_demo_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('old_demo_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_old_demo_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=1edf651b-575d-440c-b9a1-f64002be689d>

# bu_bureau_score

In [158]:
sq = """
with bu_bureau_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    bu_bureau_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    coalesce(bu_bureau_score, 0.0) > 0.0
  AND
    ln_mature_fpd10_flag = 1
)
select * from bu_bureau_score;
"""

bu_bureau_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

bu_bureau_scorefpd10.head()

gini_results = calculate_periodic_gini_threedigit(bu_bureau_scorefpd10, 'bu_bureau_score', 'ln_fpd10_flag', 'FPD10')
M1FPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M1FPD10.shape}")
M1FPD10.columns.values

# FPD30

sq = """
with bu_bureau_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    bu_bureau_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    coalesce(bu_bureau_score, 0.0) > 0.0
  AND
    ln_mature_fpd30_flag = 1
)
select * from bu_bureau_score;
"""

bu_bureau_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

bu_bureau_scorefpd30.head()

gini_results = calculate_periodic_gini_threedigit(bu_bureau_scorefpd30, 'bu_bureau_score', 'ln_fpd30_flag', 'FPD30')
M2FPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M2FPD30.shape}")
M2FPD30.columns.values

sq = """
with bu_bureau_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    bu_bureau_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    coalesce(bu_bureau_score, 0.0) > 0.0
  AND
    ln_mature_fspd30_flag = 1
)
select * from bu_bureau_score;
"""

bu_bureau_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(bu_bureau_scorefspd30.head(2))

gini_results = calculate_periodic_gini_threedigit(bu_bureau_scorefspd30, 'bu_bureau_score', 'ln_fspd30_flag', 'FSPD30')
M3FSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M3FSPD30.shape}")
print(M3FSPD30.columns.values)

M3FSPD30.head()

sq = """
with bu_bureau_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    bu_bureau_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    coalesce(bu_bureau_score, 0.0) > 0.0
  AND
    ln_mature_fstpd30_flag = 1
)
select * from bu_bureau_score;
"""

bu_bureau_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(bu_bureau_scorefstpd30.head(2))

gini_results = calculate_periodic_gini_threedigit(bu_bureau_scorefstpd30, 'bu_bureau_score', 'ln_fstpd30_flag', 'FSTPD30')
M4FSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M4FSTPD30.shape}")
print(M4FSTPD30.columns.values)

M4FSTPD30.head()



Job ID 35f89581-9aeb-4528-82fb-4c3fd2a32ae5 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(92, 7)
Job ID fc743d78-33d6-4f89-8887-d261c66e846b successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(92, 7)
Job ID ff6bd32d-5a3f-4b0d-b653-fee4bf668e53 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  bu_bureau_score  \
0 2023-01-10 16:49:53  f7506ebf-ba0b-40a4-b0bd-8ec91ea0ab96            339.0   
1 2023-01-12 17:50:30  a2fd2c0e-ad17-48e2-aac9-b6a22ad444d8            357.0   

   ln_fspd30_flag  ln_mature_fspd30_flag  
0               1                      1  
1               0                      1  
The shape of dataframe after copy is:	(92, 7)
['start_date' 'end_date' 'bu_bureau_score_FSPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']
Job ID 7fabe981-fd76-44e0-bea0-82d9c6f63fc6 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  bu_bureau_score  \
0 2023-01-10 16:49:53  f7506ebf-ba0b-40a4-b0bd-8ec91ea0ab96            339.0   
1 2023-01-12 17:50:30  a2fd2c0e-ad17-48e2-aac9-b6a22ad444d8            357.0   

   ln_fstpd30_flag  ln_mature_fstpd30_flag  
0                1                       1  
1                0                       1  
The shape of dataframe after copy is:	(92, 7)
['start_date' 'end_date' 'bu_bureau_score_FSTPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']


,start_date,end_date,bu_bureau_score_FSTPD30_gini,period,Model_Name,version,bad_rate
0,2023-01-01,2023-01-31,0.033333,Month,bu_bureau_score,1.1.0,FSTPD30
1,2023-01-09,2023-01-15,-0.080000,Week,bu_bureau_score,1.1.0,FSTPD30
2,2023-01-16,2023-01-22,-0.833333,Week,bu_bureau_score,1.1.0,FSTPD30
3,2023-01-23,2023-01-29,0.416667,Week,bu_bureau_score,1.1.0,FSTPD30
4,2023-01-30,2023-02-05,0.266667,Week,bu_bureau_score,1.1.0,FSTPD30


## Combining data

In [159]:
import functools

dataframes = [M1FPD10, M2FPD30, M3FSPD30, M4FSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'bu_bureau_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 'bu_bureau_score_FPD30_gini',
       'bu_bureau_score_FSPD30_gini', 'bu_bureau_score_FSTPD30_gini'],
      dtype=object)

In [160]:
final_df = final_df[['start_date', 'end_date', 'period',   'Model_Name', 'version', 'bad_rate','bu_bureau_score_FPD10_gini','bu_bureau_score_FPD30_gini',    'bu_bureau_score_FSPD30_gini', 'bu_bureau_score_FSTPD30_gini']].copy()

In [161]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_bu_bureau_score;"""
client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=0fbf7840-7379-4607-8286-af7ce1aa4c40>

In [162]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('bu_bureau_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('bu_bureau_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('bu_bureau_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('bu_bureau_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_bu_bureau_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=8d841bc0-48b3-40a0-92e1-47b75cd1517f>